<a href="https://colab.research.google.com/github/mansi-arora0/IMD_Project/blob/main/JetVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xarray netCDF4 h5netcdf h5py matplotlib --quiet


In [ ]:
from google.colab import files

uploaded = files.upload()
 # 2024 - 2025



In [ ]:
import xarray as xr

# Replace the file name with your uploaded one
file_path = "d1a3a20e3f4f0af74b77d6fba10275bd.nc"

ds = xr.open_dataset(file_path, decode_times=False)
ds



In [ ]:
import pandas as pd

# Convert UNIX seconds to pandas datetime64
dt = pd.to_datetime(ds.valid_time.values, unit="s")

# Attach that array as the coordinate
ds = ds.assign_coords(valid_time=("valid_time", dt))


In [ ]:
ds = ds.assign_coords(
        valid_time=("valid_time",
                    pd.to_datetime(ds.valid_time.values, unit="s"))
     )


In [ ]:
print(ds.valid_time.isel(valid_time=slice(0, 5)))


In [ ]:
# Compute wind speed (in m/s)
ws = (ds['u']**2 + ds['v']**2)**0.5

# Resample to daily average wind speed
daily_ws = ws.resample(valid_time="1D").mean()

# See the result
daily_ws


In [ ]:
daily_ws = ws.resample(valid_time="1D").mean()


In [ ]:
import matplotlib.pyplot as plt

# Select a date and pressure level
date = '2024-05-01'
level = 850  # You can also use 925

daily_ws.sel(valid_time=date, pressure_level=level).plot(
    x='longitude', y='latitude', cmap='viridis',
    figsize=(10, 6), cbar_kwargs={'label': 'Wind Speed (m/s)'}
)
plt.title(f'Daily Wind Speed on {date} at {level} hPa')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
# Select a point (e.g., New Delhi: ~28.6°N, 77.2°E)
lat = 28.5
lon = 77.25
level = 850

# Get the time series
ws_timeseries = daily_ws.sel(latitude=lat, longitude=lon, pressure_level=level, method="nearest")

# Plot it
ws_timeseries.plot(figsize=(12, 4))
plt.title(f"Wind Speed Time Series at lat={lat}, lon={lon}, level={level}")
plt.ylabel("Wind Speed (m/s)")
plt.grid(True)
plt.show()


In [ ]:
# Save processed data to NetCDF
daily_ws.to_netcdf("daily_wind_speed.nc")
print("Saved as daily_wind_speed.nc ✅")


In [ ]:
print(daily_ws.valid_time.values)
print(daily_ws.pressure_level.values)


In [ ]:
import matplotlib.pyplot as plt

# Select a date and pressure level
date  = '2024-07-15'
level = 925

daily_ws.sel(valid_time=date, pressure_level=level).plot(
    x='longitude', y='latitude', cmap='viridis',
    figsize=(10, 6), cbar_kwargs={'label': 'Wind Speed (m/s)'}
)
plt.title(f'Daily Wind Speed on {date} at {level} hPa')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
print(daily_ws.valid_time.values)
print(daily_ws.pressure_level.values)


In [ ]:
pip install pillow

In [ ]:
#Animation of Wind Speed over a Month
'''We'll create a GIF or animation showing wind speed maps day by day for 30 days'''
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.animation import FuncAnimation

# Select 30 days from your available time range
dates = daily_ws.valid_time.values[:30]  # First 30 days
level = 925  # or 850

fig, ax = plt.subplots(figsize=(10, 6))

# Create an initial plot
img = daily_ws.sel(valid_time=dates[0], pressure_level=level).plot(
    ax=ax, x='longitude', y='latitude', cmap='viridis', add_colorbar=False
)
cbar = fig.colorbar(img, ax=ax, orientation='vertical', label='Wind Speed (m/s)')
title = ax.set_title(f'Wind Speed on {str(dates[0])[:10]} at {level} hPa')

# Update function for animation
def update(i):
    data = daily_ws.sel(valid_time=dates[i], pressure_level=level)
    img.set_array(data.values.flatten())
    title.set_text(f'Wind Speed on {str(dates[i])[:10]} at {level} hPa')

# Animate
anim = FuncAnimation(fig, update, frames=len(dates), interval=500)

# Save as GIF (you need imagemagick or Pillow)
anim.save('wind_animation.gif', writer='pillow')  # or writer='imagemagick'
plt.close()


In [ ]:
import os, glob, pathlib

# List GIFs in the current working directory
print("Current working dir:", pathlib.Path().absolute())
print("GIFs I see here:", glob.glob("*.gif"))


In [ ]:
#Comparison Plot for 2 Pressure Levels on Same Date
date = '2024-06-15'

fig, axs = plt.subplots(1, 2, figsize=(15, 6))

for i, level in enumerate([925, 850]):
    data = daily_ws.sel(valid_time=date, pressure_level=level)
    data.plot(
        ax=axs[i], x='longitude', y='latitude', cmap='plasma',
        cbar_kwargs={'label': 'Wind Speed (m/s)'}
    )
    axs[i].set_title(f'Wind Speed on {date} at {level} hPa')
    axs[i].set_xlabel("Longitude")
    axs[i].set_ylabel("Latitude")

plt.tight_layout()
plt.show()


In [ ]:
#Animation of Wind Speed over a Month
'''We'll create a GIF or animation showing wind speed maps day by day for 30 days'''
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.animation import FuncAnimation

# Select 30 days from your available time range
dates = daily_ws.valid_time.values[:30]  # First 30 days
level = 850

fig, ax = plt.subplots(figsize=(10, 6))

# Create an initial plot
img = daily_ws.sel(valid_time=dates[0], pressure_level=level).plot(
    ax=ax, x='longitude', y='latitude', cmap='viridis', add_colorbar=False
)
cbar = fig.colorbar(img, ax=ax, orientation='vertical', label='Wind Speed (m/s)')
title = ax.set_title(f'Wind Speed on {str(dates[0])[:10]} at {level} hPa')

# Update function for animation
def update(i):
    data = daily_ws.sel(valid_time=dates[i], pressure_level=level)
    img.set_array(data.values.flatten())
    title.set_text(f'Wind Speed on {str(dates[i])[:10]} at {level} hPa')

# Animate
anim = FuncAnimation(fig, update, frames=len(dates), interval=500)

# Save as GIF (you need imagemagick or Pillow)
anim.save('wind1_animation.gif', writer='pillow')  # or writer='imagemagick'
plt.close()


In [ ]:
from IPython.display import Image
Image(filename="wind_animation.gif")
'''The animation shows the variation in wind speed at 925 hPa pressure level over a period of 30 days. Wind intensities change across the Indian subcontinent due to monsoon dynamics and pressure gradients'''

In [ ]:
from IPython.display import Image
Image(filename="wind1_animation.gif")


In [ ]:
#To calculate the average wind speed for your 2024–2025 data

ws_2024_2025 = daily_ws.sel(valid_time=slice("2024-01-01", "2025-12-31"))

#2. Calculate the average over time
avg_ws_2024_2025 = ws_2024_2025.mean(dim="valid_time")

#3. Save it
avg_ws_2024_2025.to_netcdf("avg_ws_2024_2025.nc")

#4. Scaler average
scalar_avg_ws = ws_2024_2025.mean().item()
print("Overall average wind speed (2024–2025):", round(scalar_avg_ws, 2), "m/s")




In [ ]:
import matplotlib.pyplot as plt

avg_ws_2024_2025.sel(pressure_level=850).plot(
    x='longitude', y='latitude', cmap='viridis',
    cbar_kwargs={'label': 'Avg Wind Speed (m/s)'},
    figsize=(10, 6)
)
plt.title("Average Wind Speed (2024–2025) at 850 hPa")
plt.show()


In [ ]:
from google.colab import files

uploaded = files.upload()
#2015 - 2016

In [ ]:
from google.colab import files

uploaded = files.upload()
#2017 - 2018


In [ ]:
from google.colab import files

uploaded = files.upload()
#2019 - 2020


In [ ]:
from google.colab import files

uploaded = files.upload()
#2021 - 2022


In [ ]:
from google.colab import files

uploaded = files.upload()
#2023

In [ ]:
import os

for file in os.listdir("/content"):
    if file.endswith(".nc"):
        print(file)


In [ ]:
from google.colab import files

uploaded = files.upload()
#2015 - 2016


In [ ]:
from google.colab import files

uploaded = files.upload()
#2024 - 2025


In [ ]:
import os

for file in os.listdir("/content"):
    if file.endswith(".nc"):
        print(file)

In [ ]:
import xarray as xr

# List your NetCDF file names (make sure these match exactly!)
file_list = [
    "/content/7fc3dc956166a8e81e8d2186fea25585.nc",   # 2015–2016
    "/content/69e2b3a22b07d506df5eaa6990c937af.nc",   # 2017–2018
    "/content/f48a038b5b49b403f49f163404f74967.nc",   # 2019–2020
    "/content/342854a6b62d23b9f43ce5792ee0e5c9.nc",   # 2021–2022
    "/content/c612badfed4c054f3fa18ef4244b20e.nc",    # 2023
    "/content/d1a3a20e3f4f0af74b77d6fba10275bd.nc",   # 2024–2025
]

# Merge all files into one dataset
combined_ds = xr.open_mfdataset(file_list, combine='by_coords')

# Check the contents
combined_ds


In [ ]:
combined_ds.data_vars



In [ ]:
#average wind speed from 2015 to 2025:

import numpy as np

# Step 1: Compute wind speed from u and v components
wind_speed = np.sqrt(combined_ds['u']**2 + combined_ds['v']**2)

# Step 2: Select data between 2015 and 2025
ws_2015_2025 = wind_speed.sel(valid_time=slice("2015-01-01", "2025-12-31"))

# Step 3: Average over time (to get spatial wind speed map)
avg_ws_2015_2025 = ws_2015_2025.mean(dim="valid_time")

# Step 4: Save to NetCDF file
avg_ws_2015_2025.to_netcdf("avg_ws_2015_2025.nc")

# Step 5: Scalar average (overall single value for all years and locations)
scalar_avg_ws = ws_2015_2025.mean().compute().item()
print("Overall average wind speed (2015–2025):", round(scalar_avg_ws, 2), "m/s")


In [ ]:
print(ws_2015_2025.valid_time.min().values)
print(ws_2015_2025.valid_time.max().values)


In [ ]:
print(f"Average wind speed from {ws_2015_2025.valid_time.min().values} to {ws_2015_2025.valid_time.max().values}: {round(scalar_avg_ws, 2)} m/s")


In [ ]:
#Calculate Average Wind Direction (2015–2025):
# Step 1: Select u and v wind components from 2015–2025

u_2015_2025 = combined_ds["u"].sel(valid_time=slice("2015-01-01", "2025-12-31"))
v_2015_2025 = combined_ds["v"].sel(valid_time=slice("2015-01-01", "2025-12-31"))


In [ ]:
#Step 2: Calculate wind direction in degrees
#Use arctangent to calculate the direction from u and v:

import numpy as np

# Calculate direction in radians, then convert to degrees
wind_dir_rad = np.arctan2(u_2015_2025, v_2015_2025)
wind_dir_deg = (np.degrees(wind_dir_rad) + 360) % 360  # Convert to degrees and normalize 0–360°


In [ ]:
#Step 3: Average over time

avg_wind_dir_2015_2025 = wind_dir_deg.mean(dim="valid_time")


In [ ]:
#Step 4: Convert to scalar average for overall value (optional)
#This gives a single overall wind direction, averaged over the entire 11-year period.

scalar_avg_dir = avg_wind_dir_2015_2025.mean().compute().item()
print(f"Average wind direction (2015–2025): {round(scalar_avg_dir, 2)} degrees")


In [ ]:
#convert average wind speed to knots

mps_speed = 7.66
knots_speed = mps_speed * 1.94384
print("Average wind speed in knots:", round(knots_speed, 2), "knots")


In [ ]:
!pip install cartopy haversine

In [ ]:
import os

for file in os.listdir("/content"):
    if file.endswith(".nc"):
        print(file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

folder_path = "/content/drive/MyDrive/wind_data"


In [ ]:
import os

for file in os.listdir("/content"):
    if file.endswith(".nc"):
        print(file)

In [ ]:
import numpy as np

# 1. Extract u and v wind components (you can adjust pressure level if needed)
u = combined_ds['u']
v = combined_ds['v']

# 2. Calculate wind speed: sqrt(u² + v²)
wind_speed = np.sqrt(u**2 + v**2)

# 3. Compute average wind speed over time
avg_wind_speed = wind_speed.mean(dim='valid_time')
print("✅ Average Wind Speed (2015–2025) calculated.")

# 4. Compute scalar average wind speed (a single number)
scalar_avg_wind_speed = float(wind_speed.mean().values)
print("🌬️ Scalar Average Wind Speed (2015–2025):", round(scalar_avg_wind_speed, 2), "m/s")

# 5. Convert to knots (1 m/s = 1.94384 knots)
scalar_avg_knots = scalar_avg_wind_speed * 1.94384
print("⛵ Scalar Average Wind Speed (in knots):", round(scalar_avg_knots, 2), "knots")

# 6. Calculate wind direction in degrees
wind_direction = (np.arctan2(u, v) * 180 / np.pi) % 360
avg_wind_direction = wind_direction.mean(dim='valid_time')
scalar_avg_direction = float(wind_direction.mean().values)
print("🧭 Scalar Average Wind Direction (2015–2025):", round(scalar_avg_direction, 2), "°")


In [ ]:
'''Define Wind Speed Regions and Calculate Area Covered
We’ll use your average wind speed (in knots) and define 3 regions:
30–40 knots and 40–50 knots and 50+ knots
Then, we’ll estimate how much area (in km²) each region covers.
 '''

#First, define the speed regions:
# 1. Convert the average wind speed to knots
avg_wind_speed_knots = avg_wind_speed * 1.94384

# 2. Define masks for each speed region
speed_regions = {
    '30-40 knots': (avg_wind_speed_knots >= 30) & (avg_wind_speed_knots < 40),
    '40-50 knots': (avg_wind_speed_knots >= 40) & (avg_wind_speed_knots < 50),
    '50+ knots': (avg_wind_speed_knots >= 50)
}

#Step 2: Estimate Area of One Grid Cell
!pip install haversine


In [ ]:
from haversine import haversine, Unit

# Use two points from your grid to estimate cell size
lat1 = float(avg_wind_speed.latitude[0])
lat2 = float(avg_wind_speed.latitude[1])
lon1 = float(avg_wind_speed.longitude[0])
lon2 = float(avg_wind_speed.longitude[1])

# Calculate width and height
width_km = haversine((lat1, lon1), (lat1, lon2), unit=Unit.KILOMETERS)
height_km = haversine((lat1, lon1), (lat2, lon1), unit=Unit.KILOMETERS)
cell_area_km2 = width_km * height_km

print(f"🧾 Estimated area of one grid cell: {cell_area_km2:.2f} km²")


In [ ]:
#Step 3: Calculate Total Area for Each Region

for label, mask in speed_regions.items():
    num_cells = int(mask.sum().compute().item())   # ← fixed
    total_area = num_cells * cell_area_km2
    print(f"\n🌪️ Wind Region: {label}")
    print(f"   ▪ Grid Cells: {num_cells}")
    print(f"   ▪ Area Covered: {total_area:.2f} km²")


In [ ]:
#Step-by-step Python code to plot wind speed regions

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Meshgrid for plotting
lon2d, lat2d = np.meshgrid(u0.longitude, u0.latitude)

# Define masks again (just for clarity)
region_30_40 = (speed_knots >= 30) & (speed_knots < 40)
region_40_50 = (speed_knots >= 40) & (speed_knots < 50)
region_50_plus = speed_knots >= 50

# Create plot
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([40, 55, -5, 15], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN)

# Plot regions
plt.contourf(lon2d, lat2d, region_30_40, levels=[0.5, 1.5], colors='yellow', alpha=0.6, transform=ccrs.PlateCarree(), label='30–40 knots')
plt.contourf(lon2d, lat2d, region_40_50, levels=[0.5, 1.5], colors='orange', alpha=0.6, transform=ccrs.PlateCarree(), label='40–50 knots')
plt.contourf(lon2d, lat2d, region_50_plus, levels=[0.5, 1.5], colors='red', alpha=0.6, transform=ccrs.PlateCarree(), label='50+ knots')

# Add title and legend
plt.title("Wind Speed Regions over Somali Coast (First Time Frame)", fontsize=14)
yellow_patch = plt.Line2D([0], [0], marker='s', color='w', label='30–40 knots', markerfacecolor='yellow', markersize=10)
orange_patch = plt.Line2D([0], [0], marker='s', color='w', label='40–50 knots', markerfacecolor='orange', markersize=10)
red_patch = plt.Line2D([0], [0], marker='s', color='w', label='50+ knots', markerfacecolor='red', markersize=10)
plt.legend(handles=[yellow_patch, orange_patch, red_patch], loc='lower left')

plt.show()


In [ ]:
# ============================================================
# 0. Mount Google Drive (skip if already mounted in your session)
# ============================================================
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ============================================================
# 1. Imports
# ============================================================
import glob, os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from haversine import haversine, Unit

# ============================================================
# 2. Load & combine all NetCDF files from your Drive folder
# ============================================================
folder_path = "/content/drive/MyDrive/wind_data"     # << adjust if your folder path differs
nc_files = sorted(glob.glob(os.path.join(folder_path, "*.nc")))
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Pick variable names (update if yours differ)
u_name = "u"      # e.g., 'u'  or 'u100'
v_name = "v"      # e.g., 'v'  or 'v100'

# ============================================================
# 3. Select your custom region  (North 30, South -15, West 75, East 30)
# ============================================================
u_reg = ds[u_name].sel(latitude=slice(30, -15), longitude=slice(30, 75))
v_reg = ds[v_name].sel(latitude=slice(30, -15), longitude=slice(30, 75))

# ============================================================
# 4. Choose pressure level (comment out if data is already 2‑D)
# ============================================================
if "pressure_level" in u_reg.dims:
    PL = 850                 # pick 850 hPa; change to 925 or 100 for 100 m if needed
    u_reg = u_reg.sel(pressure_level=PL)
    v_reg = v_reg.sel(pressure_level=PL)

# ============================================================
# 5. Select a time index (t = 0 is first date)
# ============================================================
t = 0                       # change to another index for another date
u0 = u_reg.isel(valid_time=t)
v0 = v_reg.isel(valid_time=t)

# 2‑D lon/lat grids
lon2d, lat2d = np.meshgrid(u0.longitude, u0.latitude)

# ============================================================
# 6. Wind speed (m s⁻¹) and knots
# ============================================================
speed0 = np.sqrt(u0**2 + v0**2)
speed_knots = speed0 * 1.94384   # 1 m/s = 1.94384 knots

# ============================================================
# 7. Wind direction (° from which wind blows)
# ============================================================
direction0 = (np.degrees(np.arctan2(u0, v0)) + 360) % 360

# ============================================================
# 8. Estimate area of one grid cell (km²)
# ============================================================
def grid_area_km2(lat_a, lat_b, lon_a, lon_b):
    w = haversine((lat_a, lon_a), (lat_a, lon_b), unit=Unit.KILOMETERS)
    h = haversine((lat_a, lon_a), (lat_b, lon_a), unit=Unit.KILOMETERS)
    return w * h

cell_area = grid_area_km2(float(lat2d[0,0]), float(lat2d[1,0]),
                          float(lon2d[0,0]), float(lon2d[0,1]))

# ============================================================
# 9. Speed‑region masks & area / direction stats
# ============================================================
thresholds = [30, 40, 50]           # knots
for thr in thresholds:
    mask = speed_knots > thr
    num_cells = int(mask.sum().compute().item())      # safe for Dask
    total_area = num_cells * cell_area

    if num_cells > 0:
        dirs = direction0.where(mask)
        rad  = np.deg2rad(dirs)
        mean_dir = (np.arctan2(np.nanmean(np.sin(rad)),
                               np.nanmean(np.cos(rad))) * 180/np.pi) % 360
        dir_text = f"{mean_dir:.2f}°"
    else:
        dir_text = "No data"

    print(f"\n🌪️  Wind Region > {thr} kt")
    print(f"   ▪ Grid cells : {num_cells}")
    print(f"   ▪ Area       : {total_area:,.0f} km²")
    print(f"   ▪ Avg dir    : {dir_text}")

# ============================================================
# 10. Plot wind vectors coloured by speed
# ============================================================
plt.figure(figsize=(11, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 30])
ax.coastlines(); ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND, facecolor="lightgray"); ax.add_feature(cfeature.OCEAN)

q = ax.quiver(lon2d, lat2d, u0.values, v0.values, speed0.values,
              scale=700, cmap="viridis")
plt.colorbar(q, label="Wind speed (m s⁻¹)")
date_str = str(u0.valid_time.values)[:10]
plt.title(f"Wind vectors at {PL if 'pressure_level' in u_reg.dims else 'model level'} hPa • {date_str}")
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install cartopy haversine

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from haversine import haversine, Unit
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Folder with your NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".nc")]

# Combine multiple NetCDF files
ds = xr.open_mfdataset(file_list, combine='by_coords')

# Use u and v components at 850 hPa level
pressure_level = 850
u = ds['u'].sel(pressure_level=pressure_level)
v = ds['v'].sel(pressure_level=pressure_level)

# Somali Jet region: N=15, S=-15, W=30, E=75
u_region = u.sel(latitude=slice(15, -15), longitude=slice(30, 75))
v_region = v.sel(latitude=slice(15, -15), longitude=slice(30, 75))

# Use first time frame
u0 = u_region.isel(valid_time=0)
v0 = v_region.isel(valid_time=0)
speed0 = np.sqrt(u0**2 + v0**2)

# Convert to knots
speed_knots = speed0 * 1.94384

# Estimate grid cell area in km²
def estimate_grid_area(lat1, lat2, lon1, lon2):
    p1, p2 = (lat1, lon1), (lat2, lon1)
    p3 = (lat1, lon2)
    width = haversine(p1, p3, unit=Unit.KILOMETERS)
    height = haversine(p1, p2, unit=Unit.KILOMETERS)
    return width * height

grid_area = estimate_grid_area(
    float(u0.latitude[0]), float(u0.latitude[1]),
    float(u0.longitude[0]), float(u0.longitude[1])
)

# Wind direction
direction_deg = (np.arctan2(u0, v0) * 180 / np.pi) % 360

# Analyze by speed thresholds
thresholds = [30, 40, 50]
for thresh in thresholds:
    mask = speed_knots > thresh
    num_cells = mask.sum().compute().item()
    area = num_cells * grid_area

    dir_masked = direction_deg.where(mask)
    radians = np.deg2rad(dir_masked)
    mean_sin = np.nanmean(np.sin(radians))
    mean_cos = np.nanmean(np.cos(radians))
    mean_dir = (np.arctan2(mean_sin, mean_cos) * 180 / np.pi) % 360

    print(f"\n🌪️ Wind Region: {thresh}+ knots")
    print(f"   ▪ Grid Cells: {num_cells}")
    print(f"   ▪ Area Covered: {area:.2f} km²")
    print(f"   ▪ Avg Direction: {mean_dir:.2f}°")

# Plot wind vectors
lon2d, lat2d = np.meshgrid(u0.longitude, u0.latitude)
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 15], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN)
plt.quiver(lon2d, lat2d, u0.values, v0.values, speed0.values, scale=700)
plt.colorbar(label='Wind Speed (m/s)')
plt.title(f"Wind Vectors at 850 hPa • {str(u.valid_time.values[0])[:10]}")
plt.show()


In [ ]:
'''Track and visualize how wind evolves from May–August each year using:
Wind speeds (esp. >30 knots)
Area covered
Average wind direction
Animated daily maps'''

#🔁 Step 1: Loop Over Dates & Track Key Stats
'''We will calculate for each day:
Total area covered by wind speed > 30 knots
Average direction in that region'''

import xarray as xr
import numpy as np
import pandas as pd
from haversine import haversine, Unit

# Correct your latitude and longitude slicing:
lat_range = slice(30, -15)        # From North to South
lon_range = slice(30, 75)         # From East to West


# Load your merged dataset (replace with your actual dataset)
ds = xr.open_dataset("/content/drive/MyDrive/combined_wind.nc")

# Select wind components
# Use 925 hPa or 850 hPa (whichever you prefer or want to compare)
u = ds['u'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))
v = ds['v'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))

time = ds['valid_time']

# Estimate area of one grid cell
def estimate_grid_area(lat1, lat2, lon1, lon2):
    p1 = (lat1, lon1)
    p2 = (lat2, lon1)
    p3 = (lat1, lon2)
    width = haversine(p1, p3, unit=Unit.KILOMETERS)
    height = haversine(p1, p2, unit=Unit.KILOMETERS)
    return width * height

grid_area = estimate_grid_area(
    float(u.latitude[0]), float(u.latitude[1]),
    float(u.longitude[0]), float(u.longitude[1])
)

# Time loop
records = []


for i in range(len(time)):
    u0 = u.isel(valid_time=i)
    v0 = v.isel(valid_time=i)

    speed = np.sqrt(u0**2 + v0**2)
    speed_knots = speed * 1.94384
    # Compute wind direction in degrees (meteorological: from where wind is coming)
    wind_dir = (np.degrees(np.arctan2(-u0.values, -v0.values)) + 360) % 360
    #direction_deg = (np.arctan2(u0, v0) * 180 / np.pi) % 360

    # Mask where speed > 30 knots
    mask = speed_knots > 30
    cell_count = mask.sum().item()
    area_km2 = cell_count * grid_area

    # Calculate average wind direction
    if np.sum(mask) == 0:
       mean_dir_deg = "N/A"
    else:
       radians = np.radians(wind_dir[mask])
       mean_sin = np.nanmean(np.sin(radians))
       mean_cos = np.nanmean(np.cos(radians))
       mean_dir = np.arctan2(mean_sin, mean_cos)
       mean_dir_deg = (np.degrees(mean_dir) + 360) % 360
       mean_dir_deg = round(mean_dir_deg, 2)


    records.append({
        "date": str(time[i].values)[:10],
        "area_km2": area_km2,
        "avg_direction": mean_dir
    })

df = pd.DataFrame(records)
df.to_csv("somali_jet_stats.csv", index=False)


In [ ]:
!pip install haversine


In [ ]:
import os
import glob

folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted(glob.glob(os.path.join(folder_path, "*.nc")))
print("Files found:", nc_files)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob

# Set the correct path
folder_path = "/content/drive/MyDrive/wind_data"  # <-- update this

# Check
if os.path.exists(folder_path):
    print("✅ Folder exists:", folder_path)
    print("📂 Files in folder:", os.listdir(folder_path))
    print("🔍 .nc files found:", glob.glob(os.path.join(folder_path, "*.nc")))
else:
    print("❌ Folder not found:", folder_path)


In [ ]:
import xarray as xr
import glob
import os

# Use your verified folder path here:
folder_path = "/content/drive/MyDrive/wind_data"  # Replace with actual path

# Get all .nc files
nc_files = sorted(glob.glob(os.path.join(folder_path, "*.nc")))
print(f"✅ Found {len(nc_files)} .nc files")

# Combine all datasets
combined_ds = xr.open_mfdataset(nc_files, combine='by_coords')

# (Optional) Save it for quick reuse later
combined_ds.to_netcdf("/content/drive/MyDrive/combined_wind.nc")
print("💾 Combined dataset saved as 'combined_wind.nc'")


In [ ]:
!pip install cartopy


In [ ]:
'''🎥 Goal: Create Daily Wind Vector Map Animation
We’ll:

Plot wind vectors (arrows) on a map for each day.

Show wind speed magnitude with color.

Save as a video (.mp4) or animated .gif.'''
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation

# Load combined dataset
ds = xr.open_dataset("/content/drive/MyDrive/combined_wind.nc")

# Select wind components at 925 hPa
u = ds['u'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))
v = ds['v'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))
time = ds['valid_time']

# Subsample grid (optional, for clarity in animation)
stride = 3  # Adjust for fewer/more arrows

lats = u.latitude.values[::stride]
lons = u.longitude.values[::stride]
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create figure
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, facecolor='lightgray')
quiver = None
title = ax.set_title("", fontsize=14)

# Animation function
def update(i):
    global quiver
    ax.clear()
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    u0 = u.isel(valid_time=i).values[::stride, ::stride]
    v0 = v.isel(valid_time=i).values[::stride, ::stride]
    speed = np.sqrt(u0**2 + v0**2)

    # Plot arrows
    quiver = ax.quiver(lon_grid, lat_grid, u0, v0, speed,
                       cmap='plasma', scale=700, width=0.003,
                       transform=ccrs.PlateCarree())

    ax.set_title(f"Wind Vectors | Date: {str(time[i].values)[:10]}", fontsize=14)

# Create animation
anim = FuncAnimation(fig, update, frames=len(time), interval=300)

# Save as MP4
anim.save("/content/drive/MyDrive/wind_animation.mp4", writer="ffmpeg", dpi=150)
print("✅ Animation saved to Google Drive as 'wind_animation.mp4'")

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import numpy as np
!pip install cartopy


In [ ]:
# ── 1. Imports ─────────────────────────────────────────────────────────
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.animation as animation
from IPython.display import HTML

# ── 2. Load dataset (already combined) ─────────────────────────────────
ds = xr.open_dataset("/content/drive/MyDrive/combined_wind.nc")

# 925 or 850 hPa – pick one
u_all = ds['u'].sel(pressure_level=850 , latitude=slice(30, -15),
                                        longitude=slice(30, 75))
v_all = ds['v'].sel(pressure_level=850 , latitude=slice(30, -15),
                                        longitude=slice(30, 75))
time = u_all.valid_time        # common time coordinate

# ── 3. Make a lon/lat grid (sub‑sample for clarity) ────────────────────
stride = 3                                # arrows every 3 grid‑points
lon2d, lat2d = np.meshgrid(u_all.longitude.values[::stride],
                           u_all.latitude .values[::stride])

# ── 4. Set up the figure & basemap (draw only once) ────────────────────
fig = plt.figure(figsize=(10, 8))
ax  = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND , facecolor='lightgray')
ax.add_feature(cfeature.OCEAN)

# First frame’s vectors just to initialise the artist
u0 = u_all.isel(valid_time=0).values[::stride, ::stride]
v0 = v_all.isel(valid_time=0).values[::stride, ::stride]
speed0 = np.sqrt(u0**2 + v0**2)
quiv = ax.quiver(lon2d, lat2d, u0, v0, speed0,
                 cmap='viridis', scale=700, width=0.003,
                 transform=ccrs.PlateCarree())
title = ax.set_title(f"Wind Vectors • {str(time[0].values)[:10]}")

# ── 5. Update function for animation ───────────────────────────────────
def update(frame):
    u_plot = u_all.isel(valid_time=frame).values[::stride, ::stride]
    v_plot = v_all.isel(valid_time=frame).values[::stride, ::stride]
    speed  = np.sqrt(u_plot**2 + v_plot**2)

    # update quiver object in‑place (fast!)
    quiv.set_UVC(u_plot, v_plot, speed)
    title.set_text(f"Wind Vectors • {str(time[frame].values)[:10]}")
    return quiv, title

# ── 6. Build & save animation ──────────────────────────────────────────
anim = animation.FuncAnimation(fig, update,
                               frames=len(time),
                               interval=300, blit=False)

mp4_path = "/content/drive/MyDrive/wind_animation_2015_2025.mp4"
anim.save(mp4_path, writer='ffmpeg', dpi=150)
print("✅ Animation saved:", mp4_path)

# ── 7. Show inline preview (JS animation) ──────────────────────────────
HTML(anim.to_jshtml())


In [ ]:
import xarray as xr
import os

folder_path = "/content/drive/MyDrive/wind_data"
file_list = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])

# Check files are found
if not file_list:
    print("❌ No .nc files found in:", folder_path)
else:
    print("✅ Files found:", file_list)

# Combine and save
ds = xr.open_mfdataset(file_list, combine='by_coords')
ds.to_netcdf("/content/drive/MyDrive/combined_wind.nc")
print("✅ combined_wind.nc saved to Drive")


In [ ]:
# ── 0. Install cartopy (first run only) ────────────────────────────────
# !pip install cartopy --quiet

# ── 1. Imports ────────────────────────────────────────────────────────
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.animation as animation
from IPython.display import HTML
import os, glob

# ── 2. Mount Google Drive ─────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── 3. Combine yearly files once (skip if combined_wind.nc already) ───
folder_path = "/content/drive/MyDrive/wind_data"
combined_path = "/content/drive/MyDrive/combined_wind.nc"

if not os.path.exists(combined_path):
    nc_files = sorted(glob.glob(os.path.join(folder_path, "*.nc")))
    if not nc_files:
        raise FileNotFoundError("No .nc files found in wind_data folder!")
    ds_tmp = xr.open_mfdataset(nc_files, combine='by_coords')
    ds_tmp.to_netcdf(combined_path)
    print("✅ combined_wind.nc saved")

# ── 4. Load dataset & select region/level ─────────────────────────────
ds = xr.open_dataset(combined_path)

pressure_level = 850  # choose 850 hPa
u = ds["u"].sel(pressure_level=pressure_level,
                latitude=slice(30, -15),  # N→S
                longitude=slice(30, 75))  # E→W
v = ds["v"].sel(pressure_level=pressure_level,
                latitude=slice(30, -15),
                longitude=slice(30, 75))
time = u.valid_time

# ── 5. Prep meshgrid (sub‑sample for clarity) ─────────────────────────
stride = 3
lon2d, lat2d = np.meshgrid(u.longitude.values[::stride],
                           u.latitude .values[::stride])

# ── 6. Set up figure & static basemap ─────────────────────────────────
fig = plt.figure(figsize=(10, 8))
ax  = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND , facecolor='lightgray')
ax.add_feature(cfeature.OCEAN)

# Initialise quiver with first frame
u0 = u.isel(valid_time=0).values[::stride, ::stride]
v0 = v.isel(valid_time=0).values[::stride, ::stride]
spd0 = np.sqrt(u0**2 + v0**2)
quiv = ax.quiver(lon2d, lat2d, u0, v0, spd0,
                 cmap='viridis', scale=700, width=0.003)
title = ax.set_title(f"Wind Vectors • {str(time.values[0])[:10]}")

# ── 7. Animation update function ──────────────────────────────────────
def update(frame):
    global quiv
    # Remove previous arrows safely
    for coll in list(ax.collections):
        try:
            coll.remove()
        except ValueError:
            pass
    if quiv:
        try:
            quiv.remove()
        except ValueError:
            pass

    u_plot = u.isel(valid_time=frame).values[::stride, ::stride]
    v_plot = v.isel(valid_time=frame).values[::stride, ::stride]
    spd    = np.sqrt(u_plot**2 + v_plot**2)

    quiv = ax.quiver(lon2d, lat2d, u_plot, v_plot, spd,
                     cmap='viridis', scale=700, width=0.003)
    title.set_text(f"Wind Vectors • {str(time.values[frame])[:10]}")
    return quiv, title

# ── 8. Build & save animation ─────────────────────────────────────────
anim = animation.FuncAnimation(fig, update,
                               frames=len(time),
                               interval=300)

mp4_out = "/content/drive/MyDrive/wind_animation_2015_2025.mp4"
anim.save(mp4_out, writer='ffmpeg', dpi=150)
print("🎥 Saved:", mp4_out)

# ── 9. Inline preview in notebook (JS/HTML5) ──────────────────────────
HTML(anim.to_jshtml())



In [ ]:
!pip install scikit-learn


In [ ]:
import xarray as xr
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Step 1: Load all NetCDF files from Drive
import os
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select wind components and region
u_all = ds['u'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))
v_all = ds['v'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))

# Step 3: Subsample to reduce size (every 5th lat/lon and first 30 time steps)
u_sub = u_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))
v_sub = v_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))

# Step 4: Flatten and stack data for clustering
u_flat = u_sub.stack(sample=("valid_time", "latitude", "longitude")).values
v_flat = v_sub.stack(sample=("valid_time", "latitude", "longitude")).values

# Remove NaNs
mask = ~np.isnan(u_flat) & ~np.isnan(v_flat)
u_clean = u_flat[mask]
v_clean = v_flat[mask]

X = np.column_stack([u_clean, v_clean])  # shape: (n_samples, 2)

# Step 5: Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Step 6: Calculate average speed & direction of each cluster
print("📊 Cluster Summary:")
for i, (u_c, v_c) in enumerate(centroids):
    speed = np.sqrt(u_c**2 + v_c**2)
    direction_rad = np.arctan2(u_c, v_c)
    direction_deg = (np.degrees(direction_rad) + 360) % 360
    print(f"🌀 Cluster {i+1}:")
    print(f"   ▪ Mean Speed: {speed:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

# Optional: visualize
plt.figure(figsize=(6,6))
plt.quiver([0]*len(centroids), [0]*len(centroids), centroids[:, 0], centroids[:, 1],
           angles='xy', scale_units='xy', scale=1, color='r')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.title("Wind Cluster Centers (Vector Plot)")
plt.xlabel("U component (m/s)")
plt.ylabel("V component (m/s)")
plt.grid()
plt.show()

In [ ]:
import xarray as xr
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Step 1: Load all NetCDF files from Drive
import os
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select wind components and region
u_all = ds['u'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))
v_all = ds['v'].sel(pressure_level=925, latitude=slice(30, -15), longitude=slice(30, 75))

# Step 3: Subsample to reduce size (every 5th lat/lon and first 30 time steps)
u_sub = u_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))
v_sub = v_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))

# Step 4: Flatten and stack data for clustering
u_flat = u_sub.stack(sample=("valid_time", "latitude", "longitude")).values
v_flat = v_sub.stack(sample=("valid_time", "latitude", "longitude")).values

# Remove NaNs
mask = ~np.isnan(u_flat) & ~np.isnan(v_flat)
u_clean = u_flat[mask]
v_clean = v_flat[mask]

X = np.column_stack([u_clean, v_clean])  # shape: (n_samples, 2)

# Step 5: Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Step 6: Calculate average speed & direction of each cluster
print("📊 Cluster Summary:")
for i, (u_c, v_c) in enumerate(centroids):
    speed = np.sqrt(u_c**2 + v_c**2)
    direction_rad = np.arctan2(u_c, v_c)
    direction_deg = (np.degrees(direction_rad) + 360) % 360
    print(f"🌀 Cluster {i+1}:")
    print(f"   ▪ Mean Speed: {speed:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

# Optional: visualize
plt.figure(figsize=(6,6))
plt.quiver([0]*len(centroids), [0]*len(centroids), centroids[:, 0], centroids[:, 1],
           angles='xy', scale_units='xy', scale=1, color='r')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.title("Wind Cluster Centers (Vector Plot)")
plt.xlabel("U component (m/s)")
plt.ylabel("V component (m/s)")
plt.grid()
plt.show()

In [ ]:
import xarray as xr
import os

folder_path = "/content/drive/MyDrive/wind_data"
file_list = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])

# Check files are found
if not file_list:
    print("❌ No .nc files found in:", folder_path)
else:
    print("✅ Files found:", file_list)

# Combine and save
ds = xr.open_mfdataset(file_list, combine='by_coords')
ds.to_netcdf("/content/drive/MyDrive/combined_wind.nc")
print("✅ combined_wind.nc saved to Drive")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob

# Set the correct path
folder_path = "/content/drive/MyDrive/wind_data"  # <-- update this

# Check
if os.path.exists(folder_path):
    print("✅ Folder exists:", folder_path)
    print("📂 Files in folder:", os.listdir(folder_path))
    print("🔍 .nc files found:", glob.glob(os.path.join(folder_path, "*.nc")))
else:
    print("❌ Folder not found:", folder_path)

In [ ]:
!pip install cartopy

In [ ]:
import os
file_list = ["/content/drive/MyDrive/wind_data/" + f for f in os.listdir("/content/drive/MyDrive/wind_data") if f.endswith(".nc")]
ds = xr.open_mfdataset(file_list, combine='by_coords')
u = ds['u'].sel(pressure_level=850)
v = ds['v'].sel(pressure_level=850)


In [ ]:
print("labels.shape =", labels.shape)


In [ ]:
print("Shape of dominant_cluster:", dominant_cluster.shape)


In [ ]:
print("lat2d shape:", lat2d.shape)
print("lon2d shape:", lon2d.shape)
print("dominant_cluster shape:", dominant_cluster.shape)

In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from scipy.stats import mode

# Step 1: Load all NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select wind components and Somali Jet region
u_all = ds['u'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))
v_all = ds['v'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))

# Step 3: Subsample to reduce data size (every 5th point in lat/lon, and first 30 time steps)
u_sub = u_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))
v_sub = v_all.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5), valid_time=slice(0, 30))

# Step 4: Flatten for clustering
u_flat = u_sub.stack(sample=("valid_time", "latitude", "longitude")).values
v_flat = v_sub.stack(sample=("valid_time", "latitude", "longitude")).values

# Remove NaNs
mask = ~np.isnan(u_flat) & ~np.isnan(v_flat)
u_clean = u_flat[mask]
v_clean = v_flat[mask]

X = np.column_stack([u_clean, v_clean])  # shape: (n_samples, 2)

# Step 5: Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Step 6: Print summary of clusters
print("📊 Cluster Summary:")
for i, (u_c, v_c) in enumerate(centroids):
    speed = np.sqrt(u_c**2 + v_c**2)
    direction_rad = np.arctan2(u_c, v_c)
    direction_deg = (np.degrees(direction_rad) + 360) % 360
    print(f"🌀 Cluster {i+1}:")
    print(f"   ▪ Mean Speed: {speed:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

# Step 7: Reshape labels to 3D and find dominant cluster at each grid cell
n_time = u_sub.valid_time.size
n_lat = u_sub.latitude.size
n_lon = u_sub.longitude.size

labels_3d = labels.reshape(n_time, n_lat, n_lon)  # (time, lat, lon)
dominant_cluster = mode(labels_3d, axis=0, nan_policy='omit').mode
dominant_cluster = np.squeeze(dominant_cluster)  # shape: (lat, lon)

# Step 8: Plot dominant cluster map
lat = u_sub.latitude.values
lon = u_sub.longitude.values
lon2d, lat2d = np.meshgrid(lon, lat)

fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN)

cf = ax.pcolormesh(lon2d, lat2d, dominant_cluster, cmap='tab10', shading='auto')
plt.colorbar(cf, label="Cluster ID")
plt.title("🌀 Dominant Wind Cluster per Grid Cell (2015–2025)")
plt.show()
plt.savefig("/content/drive/MyDrive/wind_cluster_map.png", dpi=300)

In [ ]:
!pip install haversine

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine, Unit
import os
import pandas as pd

# Load data (already merged all .nc files earlier)
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine='by_coords')

# Select wind components
u = ds['u'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))
v = ds['v'].sel(pressure_level=850, latitude=slice(30, -15), longitude=slice(30, 75))

# Estimate area of one grid cell
def estimate_grid_area(lat1, lat2, lon1, lon2):
    p1 = (lat1, lon1)
    p2 = (lat2, lon1)
    p3 = (lat1, lon2)
    width = haversine(p1, p3, unit=Unit.KILOMETERS)
    height = haversine(p1, p2, unit=Unit.KILOMETERS)
    return width * height

grid_area = estimate_grid_area(
    float(u.latitude[0]), float(u.latitude[1]),
    float(u.longitude[0]), float(u.longitude[1])
)

# Time variable
time = ds['valid_time'].values

# Thresholds
thresholds = [20, 30, 40, 50]
area_data = {thresh: [] for thresh in thresholds}

# Precompute speed in knots (shape: time x lat x lon)
speed_knots_all = (np.sqrt(u**2 + v**2) * 1.94384).compute()

# Loop over time
for i in range(speed_knots_all.shape[0]):
    speed_frame = speed_knots_all[i]

    for thresh in thresholds:
        mask = speed_frame > thresh
        cell_count = mask.sum().item()  # ✅ Now safe, it's NumPy not Dask
        area_km2 = cell_count * grid_area
        area_data[thresh].append(area_km2)

# Convert to DataFrame
df_area = pd.DataFrame(area_data)
df_area['date'] = pd.to_datetime(time)
df_area.set_index('date', inplace=True)

# 📈 Plotting
plt.figure(figsize=(12, 6))
for thresh in thresholds:
    plt.plot(df_area.index, df_area[thresh], label=f"{thresh}+ knots")

plt.xlabel("Date")
plt.ylabel("Area (km²)")
plt.title("🌪️ Wind Area Coverage (Above Speed Thresholds)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
plt.savefig("/content/drive/MyDrive/wind_speed_area_graph.png", dpi=300)


In [ ]:
'''🔹 Pick one specific date (e.g., '2017-06-15')
🔹 Select only grid points where wind speed > 20 knots
🔹 Perform clustering (KMeans) only on that data
🔹 Plot the cluster map for that da '''

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load all NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select target date
target_date = "2017-06-15"
ds_day = ds.sel(valid_time=target_date, pressure_level=850,
                latitude=slice(30, -15), longitude=slice(30, 75))

u = ds_day['u']
v = ds_day['v']

# Step 3: Subsample to reduce size
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))

# Step 4: Compute wind speed in knots and mask by > 20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384  # m/s to knots
mask = speed_knots > 20

# Step 5: Flatten and filter valid wind vectors
u_flat = u_sub.values[mask]
v_flat = v_sub.values[mask]

X = np.column_stack([u_flat, v_flat])  # shape (n_samples, 2)

# Step 6: Cluster (if enough points)
if len(X) < 10:
    print("❗ Not enough points with > 20 knots wind on", target_date)
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # Step 7: Plot the clusters
    print(f"✅ Clustering done for {len(X)} points on {target_date}")

    plt.figure(figsize=(6, 6))
    plt.quiver([0]*len(X), [0]*len(X), X[:, 0], X[:, 1], angles='xy', scale_units='xy', scale=1,
               color=[['r', 'g', 'b'][l] for l in labels])
    plt.quiver([0]*3, [0]*3, centroids[:, 0], centroids[:, 1], color='k', scale=1.5, width=0.01)
    plt.xlim(-30, 30)
    plt.ylim(-30, 30)
    plt.grid()
    plt.title(f"💨 Wind Clusters on {target_date} (>20 knots)")
    plt.xlabel("U-component (m/s)")
    plt.ylabel("V-component (m/s)")
    plt.gca().set_aspect('equal')
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
#Plot Wind Clusters (>20 knots) on a map for one specific date - 850 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=850,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 20

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 20 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>20 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
#Plot Wind Clusters (>30 knots) on a map for one specific date - 850 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=850,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 20 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()

In [ ]:
#Plot Wind Clusters (>40 knots) on a map for one specific date - 850 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=850,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 40

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 40 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>40 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()

In [ ]:
#Plot Wind Clusters (>20 knots) on a map for one specific date - 925 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=925,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 20

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 20 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")


    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>20 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()

In [ ]:
#Plot Wind Clusters (>30 knots) on a map for one specific date - 925 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=925,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 30 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")


    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()

In [ ]:
#Plot Wind Clusters (>40 knots) on a map for one specific date - 925 pressure level

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")


# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(pressure_level=925,
                latitude=slice(30, -15), longitude=slice(30, 75))

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)


# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >20 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 40

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
# Remove the time dimension to make them 2D
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values  # shape: (lat, lon)

# Apply mask properly
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]


# Step 6: Run KMeans
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 20 knots on this date.")
else:
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction per Cluster:")

    for i in range(3):  # or change to `range(kmeans.n_clusters)` if general
    # Get all u and v values for this cluster
       u_cluster = u_masked[labels == i]
       v_cluster = v_masked[labels == i]

    # Compute mean U and V components
       u_mean = np.mean(u_cluster)
       v_mean = np.mean(v_cluster)

    # Compute average speed (magnitude)
       speed_mean = np.sqrt(u_mean**2 + v_mean**2)

       # Compute average direction in degrees (0° = North, 90° = East)
       direction_rad = np.arctan2(u_mean, v_mean)  # note: u is x, v is y
       direction_deg = (np.degrees(direction_rad) + 360) % 360

       print(f"🌀 Cluster {i+1}:")
       print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
       print(f"   ▪ Mean Direction: {direction_deg:.2f}°")


    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = ['red', 'blue', 'green']
    for i in range(3):
        idx = (labels == i)
        ax.quiver(lon_vals[idx], lat_vals[idx], u_masked[idx], v_masked[idx],
                  color=colors[i], scale=400, width=0.002, transform=ccrs.PlateCarree(),
                  label=f'Cluster {i+1}')

    plt.title(f"🌀 Wind Clusters on {target_date} (>40 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob

# Set the correct path
folder_path = "/content/drive/MyDrive/wind_data"  # <-- update this

# Check
if os.path.exists(folder_path):
    print("✅ Folder exists:", folder_path)
    print("📂 Files in folder:", os.listdir(folder_path))
    print("🔍 .nc files found:", glob.glob(os.path.join(folder_path, "*.nc")))
else:
    print("❌ Folder not found:", folder_path)

In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
from cartopy import feature as cfeature

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)

# Extract full lat/lon arrays
lat = ds_day.latitude.values
lon = ds_day.longitude.values

# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >30 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')

# Remove the time dimension
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values

# Mask points >30 knots
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Step 5b: Remove points over land
# We'll use Cartopy's LAND geometry to exclude land points

land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

# Function to test if a point is on land
def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

# Apply land mask
is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Keep only sea points
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Step 6: Run KMeans (only 1 cluster)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points over sea with wind > 30 knots on this date.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)
    labels = kmeans.labels_

    print("📐 Average Speed and Direction for Sea Region:")

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print(f"🌀 Cluster:")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Plot on map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    # Plot vectors
    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster'
    )

    plt.title(f"🌀 Wind Vectors over Sea on {target_date} (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import DBSCAN
from shapely.geometry import Point
from cartopy import feature as cfeature

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Find index of target date
time_index = np.where(ds_day['valid_time'].dt.strftime('%Y-%m-%d') == target_date)[0][0]

# Select single time slice
u = ds_day['u'].isel(valid_time=time_index)
v = ds_day['v'].isel(valid_time=time_index)

# Extract lat/lon arrays
lat = ds_day.latitude.values
lon = ds_day.longitude.values

# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute wind speed in knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare masked vectors and coordinates
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')

u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values

lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Step 5b: Remove points over land
# Use cartopy land geometries
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geom in land_geom.geometries():
        if geom.contains(point):
            return True
    return False

# Check each point
is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Keep only sea points
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Step 6: Run DBSCAN
# We'll cluster wind vectors [u, v]
X = np.column_stack((u_masked, v_masked))

if len(X) < 5:
    print("❗ Not enough ocean points with wind > 30 knots on this date.")
else:
    dbscan = DBSCAN(eps=2.5, min_samples=5).fit(X)
    labels = dbscan.labels_

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"✅ Found {n_clusters} clusters over sea.")

    # Step 7: Print cluster summaries
    for cluster_id in range(n_clusters):
        idx = labels == cluster_id
        u_cluster = u_masked[idx]
        v_cluster = v_masked[idx]
        avg_u = np.mean(u_cluster)
        avg_v = np.mean(v_cluster)
        speed_mean = np.sqrt(avg_u**2 + avg_v**2)
        direction_rad = np.arctan2(avg_u, avg_v)
        direction_deg = (np.degrees(direction_rad) + 360) % 360

        print(f"🌀 Cluster {cluster_id+1}:")
        print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
        print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 8: Plot clusters on a map
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    colors = plt.cm.get_cmap('tab10', n_clusters)

    for i in range(n_clusters):
        idx = labels == i
        ax.quiver(
            lon_vals[idx], lat_vals[idx],
            u_masked[idx], v_masked[idx],
            color=colors(i),
            scale=400, width=0.002,
            transform=ccrs.PlateCarree(),
            label=f"Cluster {i+1}"
        )

    # Plot noise points (optional)
    if np.any(labels == -1):
        idx = labels == -1
        ax.quiver(
            lon_vals[idx], lat_vals[idx],
            u_masked[idx], v_masked[idx],
            color='gray',
            scale=400, width=0.002,
            transform=ccrs.PlateCarree(),
            label='Noise'
        )

    plt.title(f"🌀 DBSCAN Wind Clusters over Sea on {target_date} (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/dbscan_wind_cluster_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
!pip install cartopy

In [ ]:
# 3 utc at >30 knots
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
from cartopy import feature as cfeature

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Filter valid_time at 03:00 UTC
import pandas as pd
target_datetime = pd.to_datetime(target_date + " 03:00:00")
ds_day_3utc = ds_day.sel(valid_time=target_datetime, method="nearest")

# Select u, v
u = ds_day_3utc['u']
v = ds_day_3utc['v']

# Extract full lat/lon arrays
lat = ds_day.latitude.values
lon = ds_day.longitude.values

# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >30 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values

# Mask points >30 knots
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Step 5b: Remove points over land
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Keep only sea points
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Step 6: KMeans Clustering (optional – only 1 cluster here)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points over sea with wind > 30 knots at 3 UTC.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)

    # Wind statistics
    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("📐 Average Speed and Direction for Sea Region at 3 UTC:")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Calculate area
    # Approx area per grid cell (0.25° x 0.25°) ~ 770 km² (rough avg near equator)
    area_per_cell_km2 = 770
    sea_cell_count = len(lat_vals)
    total_area_km2 = sea_cell_count * area_per_cell_km2
    print(f"🌊 Total Sea Area with Wind > 30 knots at 3 UTC: {total_area_km2:.2f} km²")

    # Step 8: Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster >30kt'
    )

    plt.title(f"🌀 Wind Vectors at 850 hPa on {target_date} at 3 UTC (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}_3UTC.png", dpi=300)
    plt.show()


In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
from cartopy import feature as cfeature

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and region
target_date = "2017-06-15"
ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Filter valid_time at 03:00 UTC
import pandas as pd
target_datetime = pd.to_datetime(target_date + " 03:00:00")
ds_day_3utc = ds_day.sel(valid_time=target_datetime, method="nearest")

# Select u, v
u = ds_day_3utc['u']
v = ds_day_3utc['v']

# Extract full lat/lon arrays
lat = ds_day.latitude.values
lon = ds_day.longitude.values

# Step 3: Subsample (every 5th point)
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute speed in knots and mask by >30 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 20

# Step 5: Prepare coordinates and vectors
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values

# Mask points >30 knots
lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Step 5b: Remove points over land
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Keep only sea points
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Step 6: KMeans Clustering (optional – only 1 cluster here)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points over sea with wind > 20 knots at 3 UTC.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)

    # Wind statistics
    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("📐 Average Speed and Direction for Sea Region at 3 UTC:")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Step 7: Calculate area
    # Approx area per grid cell (0.25° x 0.25°) ~ 770 km² (rough avg near equator)
    area_per_cell_km2 = 770
    sea_cell_count = len(lat_vals)
    total_area_km2 = sea_cell_count * area_per_cell_km2
    print(f"🌊 Total Sea Area with Wind > 20 knots at 3 UTC: {total_area_km2:.2f} km²")

    # Step 8: Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster >30kt'
    )

    plt.title(f"🌀 Wind Vectors at 850 hPa on {target_date} at 3 UTC (>20 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_cluster_map_{target_date}_3UTC.png", dpi=300)
    plt.show()

In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
import pandas as pd

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and 12 UTC time
target_date = "2017-06-15"
target_datetime = pd.to_datetime(target_date + " 12:00:00")

ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Use nearest time to 12 UTC
ds_12utc = ds_day.sel(valid_time=target_datetime, method="nearest")

# Extract u, v components
u = ds_12utc['u']
v = ds_12utc['v']

# Extract lat/lon
lat = ds_12utc.latitude.values
lon = ds_12utc.longitude.values

# Subsample every 5th point
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Calculate speed in knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 20

# Prepare meshgrid
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')

# Apply mask
u_vals = u_sub.values
v_vals = v_sub.values
mask2d = mask.values

lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Remove land points using Cartopy geometries
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Filter to sea only
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Area Calculation: Estimate grid cell area (approximate)
lat_resolution = abs(lat_sub[1] - lat_sub[0])
lon_resolution = abs(lon_sub[1] - lon_sub[0])
earth_radius_km = 6371

# Approximate cell area in km² using spherical geometry
cell_area_km2 = (111 * lat_resolution) * (111 * lon_resolution * np.cos(np.radians(lat_vals))).mean()
total_area_km2 = len(lat_vals) * cell_area_km2

# Print area
print(f"📏 Estimated Area Covered by Wind >20 knots over Sea (at 12 UTC): {total_area_km2:.2f} km²")

# Step 6: Run KMeans (1 cluster for now)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind >20 knots.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)
    labels = kmeans.labels_

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("🌀 Cluster Info (Sea, >20 knots):")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster'
    )

    plt.title(f"🌀 Wind Vectors over Sea on {target_date} 12 UTC (>20 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_12UTC_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
import pandas as pd

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and 12 UTC time
target_date = "2017-06-15"
target_datetime = pd.to_datetime(target_date + " 12:00:00")

ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Use nearest time to 12 UTC
ds_12utc = ds_day.sel(valid_time=target_datetime, method="nearest")

# Extract u, v components
u = ds_12utc['u']
v = ds_12utc['v']

# Extract lat/lon
lat = ds_12utc.latitude.values
lon = ds_12utc.longitude.values

# Subsample every 5th point
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Calculate speed in knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Prepare meshgrid
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')

# Apply mask
u_vals = u_sub.values
v_vals = v_sub.values
mask2d = mask.values

lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Remove land points using Cartopy geometries
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Filter to sea only
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Area Calculation: Estimate grid cell area (approximate)
lat_resolution = abs(lat_sub[1] - lat_sub[0])
lon_resolution = abs(lon_sub[1] - lon_sub[0])
earth_radius_km = 6371

# Approximate cell area in km² using spherical geometry
cell_area_km2 = (111 * lat_resolution) * (111 * lon_resolution * np.cos(np.radians(lat_vals))).mean()
total_area_km2 = len(lat_vals) * cell_area_km2

# Print area
print(f"📏 Estimated Area Covered by Wind >20 knots over Sea (at 12 UTC): {total_area_km2:.2f} km²")

# Step 6: Run KMeans (1 cluster for now)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind >30 knots.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)
    labels = kmeans.labels_

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("🌀 Cluster Info (Sea, >30 knots):")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster'
    )

    plt.title(f"🌀 Wind Vectors over Sea on {target_date} 12 UTC (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_12UTC_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
import pandas as pd

# Step 1: Load NetCDF files
folder_path = "/content/drive/MyDrive/wind_data"
nc_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")])
ds = xr.open_mfdataset(nc_files, combine="by_coords")

# Step 2: Select one date and 12 UTC time
target_date = "2017-06-15"
target_datetime = pd.to_datetime(target_date + " 12:00:00")

ds_day = ds.sel(
    pressure_level=850,
    latitude=slice(30, -15),
    longitude=slice(30, 75)
)

# Use nearest time to 12 UTC
ds_12utc = ds_day.sel(valid_time=target_datetime, method="nearest")

# Extract u, v components
u = ds_12utc['u']
v = ds_12utc['v']

# Extract lat/lon
lat = ds_12utc.latitude.values
lon = ds_12utc.longitude.values

# Subsample every 5th point
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Calculate speed in knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 40

# Prepare meshgrid
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')

# Apply mask
u_vals = u_sub.values
v_vals = v_sub.values
mask2d = mask.values

lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Remove land points using Cartopy geometries
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '50m')

def is_land(lat, lon):
    point = Point(lon, lat)
    for geometry in land_geom.geometries():
        if geometry.contains(point):
            return True
    return False

is_sea = np.array([not is_land(lat, lon) for lat, lon in zip(lat_vals, lon_vals)])

# Filter to sea only
lat_vals = lat_vals[is_sea]
lon_vals = lon_vals[is_sea]
u_masked = u_masked[is_sea]
v_masked = v_masked[is_sea]

# Area Calculation: Estimate grid cell area (approximate)
lat_resolution = abs(lat_sub[1] - lat_sub[0])
lon_resolution = abs(lon_sub[1] - lon_sub[0])
earth_radius_km = 6371

# Approximate cell area in km² using spherical geometry
cell_area_km2 = (111 * lat_resolution) * (111 * lon_resolution * np.cos(np.radians(lat_vals))).mean()
total_area_km2 = len(lat_vals) * cell_area_km2

# Print area
print(f"📏 Estimated Area Covered by Wind >40 knots over Sea (at 12 UTC): {total_area_km2:.2f} km²")

# Step 6: Run KMeans (1 cluster for now)
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind >40 knots.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)
    labels = kmeans.labels_

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("🌀 Cluster Info (Sea, >40 knots):")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    # Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='Sea Cluster'
    )

    plt.title(f"🌀 Wind Vectors over Sea on {target_date} 12 UTC (>40 knots)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/wind_12UTC_{target_date}.png", dpi=300)
    plt.show()


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.cluster import KMeans
from shapely.geometry import Point
import pandas as pd
import os

# Load the uploaded NetCDF file
file_path = "51a782427669b5c095dc96623e79a5ae.nc.nc"
ds = xr.open_dataset(file_path)

# Print available times for reference
print("Available times:", ds.valid_time.values)

# ✅ Use correct year and time from your dataset
target_date = "2015-06-21"
target_datetime = pd.to_datetime(target_date + " 12:00:00")

# Step 1: Select nearest time FIRST
ds_time = ds.sel(
    valid_time=target_datetime,
    method="nearest"
)

# Step 2: Slice latitude and longitude
ds_day = ds_time.sel(
    latitude=slice(30, -15),   # southward slice
    longitude=slice(30, 75)    # eastward slice
)

# ✅ Use the correct variable names in your dataset
u = ds_day['u10']
v = ds_day['v10']
lat = ds_day.latitude.values
lon = ds_day.longitude.values

# Step 3: Subsample every 5th point
u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
lat_sub = lat[::5]
lon_sub = lon[::5]

# Step 4: Compute wind speed in knots and mask >30 knots
speed = np.sqrt(u_sub**2 + v_sub**2)
speed_knots = speed * 1.94384
mask = speed_knots > 30

# Step 5: Prepare coordinates and mask values
lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
u_vals = u_sub.squeeze().values
v_vals = v_sub.squeeze().values
mask2d = mask.squeeze().values

lat_vals = lat2d[mask2d]
lon_vals = lon2d[mask2d]
u_masked = u_vals[mask2d]
v_masked = v_vals[mask2d]

# Step 6: Skip land filtering for speed (optional)
# Step 7: KMeans clustering
X = np.column_stack((u_masked, v_masked))
if len(X) < 10:
    print("❗ Not enough data points with wind > 30 knots at 12 UTC.")
else:
    kmeans = KMeans(n_clusters=1, random_state=0).fit(X)

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean = np.sqrt(u_mean**2 + v_mean**2)
    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    print("📐 Average Speed and Direction at 12 UTC:")
    print(f"   ▪ Mean Speed: {speed_mean:.2f} m/s")
    print(f"   ▪ Mean Direction: {direction_deg:.2f}°")

    area_per_cell_km2 = 770
    total_area_km2 = len(lat_vals) * area_per_cell_km2
    print(f"🌊 Total Area with Wind > 30 knots at 12 UTC: {total_area_km2:.2f} km²")

    # Step 8: Plot
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([30, 75, -15, 30], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN)

    ax.quiver(
        lon_vals, lat_vals,
        u_masked, v_masked,
        color='red', scale=400, width=0.002,
        transform=ccrs.PlateCarree(),
        label='>30kt Wind Vectors'
    )

    plt.title(f"🌀 Wind Vectors on {target_date} nearest 12 UTC (>30 knots)")
    plt.legend()
    plt.tight_layout()
    plt.show()


**iske baad mera kaam h -Arshleen**

In [ ]:
# Do this for every year and combine the files
!pip install netCDF4 xarray tqdm

# 📁 Upload the .nc file from local storage
from google.colab import files
uploaded = files.upload()

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import time

# 📂 Load uploaded file
filename = list(uploaded.keys())[0]
ds = xr.open_dataset(filename, engine="netcdf4")

# Extract all available times
times = pd.to_datetime(ds.valid_time.values)
dates = sorted(set([t.date() for t in times]))

# ✅ Filter by year
target_year = 2007
dates = [d for d in dates if d.year == target_year]

# Define thresholds in knots
thresholds = [20, 30, 40]

# Initialize result containers
data_3utc = {thr: [] for thr in thresholds}
data_12utc = {thr: [] for thr in thresholds}

# 📊 Function to compute wind stats
def compute_stats(ds_slice, threshold_knots):
    if 'u' not in ds_slice or 'v' not in ds_slice:
        raise KeyError("❌ Variables 'u' or 'v' not found in dataset.")

    pressure_level = 850  # Update if necessary
    u = ds_slice['u'].sel(pressure_level=pressure_level)
    v = ds_slice['v'].sel(pressure_level=pressure_level)

    lat = ds_slice.latitude.values
    lon = ds_slice.longitude.values

    u_sub = u.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
    v_sub = v.isel(latitude=slice(0, None, 5), longitude=slice(0, None, 5))
    lat_sub = lat[::5]
    lon_sub = lon[::5]

    speed = np.sqrt(u_sub ** 2 + v_sub ** 2)
    speed_knots = speed * 1.94384

    mask = speed_knots > threshold_knots

    lat2d, lon2d = np.meshgrid(lat_sub, lon_sub, indexing='ij')
    u_vals = u_sub.squeeze().values
    v_vals = v_sub.squeeze().values
    mask2d = mask.squeeze().values

    u_masked = u_vals[mask2d]
    v_masked = v_vals[mask2d]
    lat_vals = lat2d[mask2d]

    if len(u_masked) < 5:
        return None

    u_mean = np.mean(u_masked)
    v_mean = np.mean(v_masked)
    speed_mean_mps = np.sqrt(u_mean ** 2 + v_mean ** 2)
    speed_mean_knots = speed_mean_mps * 1.94384

    direction_rad = np.arctan2(u_mean, v_mean)
    direction_deg = (np.degrees(direction_rad) + 360) % 360

    area_deg2 = len(lat_vals) * (5 * 5)
    return speed_mean_knots, direction_deg, area_deg2

# 🕒 Start timer
start_time = time.time()

# 🔁 Loop through dates
for i, date in enumerate(tqdm(dates, desc=f"Processing {target_year}")):
    for utc_hour, store_dict in [(3, data_3utc), (12, data_12utc)]:
        target_dt = pd.to_datetime(f"{date} {utc_hour:02d}:00:00")
        if target_dt not in times:
            continue

        ds_time = ds.sel(valid_time=target_dt, method="nearest")
        ds_day = ds_time.sel(latitude=slice(30, -15), longitude=slice(30, 75))

        for thr in thresholds:
            stats = compute_stats(ds_day, thr)
            if stats:
                speed_mean_knots, direction_deg, area_deg2 = stats
                store_dict[thr].append({
                    "Date": date,
                    "UTC_Hour": utc_hour,
                    "Threshold_knots": thr,
                    "Mean_Speed_knots": speed_mean_knots,
                    "Mean_Direction_deg": direction_deg,
                    "Area_deg2": area_deg2
                })

    # 💾 Save partial every 100 days
    if (i + 1) % 100 == 0:
        temp_dfs = []
        for hour_label, data_dict in [("3UTC", data_3utc), ("12UTC", data_12utc)]:
            for thr, records in data_dict.items():
                if records:
                    df = pd.DataFrame(records)
                    df["UTC_Label"] = hour_label
                    temp_dfs.append(df)
        if temp_dfs:
            partial_df = pd.concat(temp_dfs, ignore_index=True)
            partial_filename = f"wind_partial_{target_year}_day{i+1}.csv"
            partial_df.to_csv(partial_filename, index=False)
            print(f"💾 Saved partial: {partial_filename}")

# ✅ Final save
dfs = []
for hour_label, data_dict in [("3UTC", data_3utc), ("12UTC", data_12utc)]:
    for thr, records in data_dict.items():
        if records:
            df = pd.DataFrame(records)
            df["UTC_Label"] = hour_label
            dfs.append(df)

if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    output_csv = f"wind_analysis_{target_year}.csv"
    final_df.to_csv(output_csv, index=False)
    print(f"\n✅ Final saved: {output_csv}")

    # 📈 Print averages
    for thr in thresholds:
        for utc_label in ["3UTC", "12UTC"]:
            sub = final_df[(final_df["Threshold_knots"] == thr) &
                           (final_df["UTC_Label"] == utc_label)]
            if not sub.empty:
                print(f"\n🌟 {utc_label} | Threshold {thr} knots:")
                print(f"▪ Mean Speed: {sub['Mean_Speed_knots'].mean():.2f} knots")
                print(f"▪ Mean Direction: {sub['Mean_Direction_deg'].mean():.2f}°")
                print(f"▪ Mean Area: {sub['Area_deg2'].mean():.2f} deg²")
else:
    print("⚠️ No data found.")

# ⏱️ Runtime
end_time = time.time()
print(f"\n⏱️ Total runtime: {round((end_time - start_time) / 60, 2)} minutes")


In [ ]:
import pandas as pd

# Combine yearly CSVs into one
combined_df = pd.concat([pd.read_csv(f"wind_analysis_{y}.csv") for y in range(1992, 2024)])
combined_df.to_csv("(1992-2024)", index=False)

print("✅ Combined file saved as '(1992-2024).csv'")

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
#plots the graphs
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

# Upload the CSV file
uploaded = files.upload()

# Assuming the file is named (1992-2014).csv
input_csv = "(1992-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)
print("📊 DataFrame shape:", df.shape)
print("🧾 Columns:", df.columns)

# Thresholds to plot
thresholds = [20, 30, 40]

# Plot area over time
for thr in thresholds:
    plt.figure(figsize=(12, 5))
    for utc_label in ["3UTC", "12UTC"]:
        sub = df[(df["Threshold_knots"] == thr) &
                 (df["UTC_Label"] == utc_label)]
        if not sub.empty:
            sub_sorted = sub.sort_values("Date")
            plt.plot(sub_sorted["Date"], sub_sorted["Area_deg2"],
                     label=f"{utc_label}", marker='o', linestyle='-')
    plt.title(f"Area Above {thr} knots Over Time (1992–2014)")
    plt.ylabel("Area (deg²)")
    plt.xlabel("Date")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

# Plot mean wind direction over time
for thr in thresholds:
    plt.figure(figsize=(12, 5))
    for utc_label in ["3UTC", "12UTC"]:
        sub = df[(df["Threshold_knots"] == thr) &
                 (df["UTC_Label"] == utc_label)]
        if not sub.empty:
            sub_sorted = sub.sort_values("Date")
            plt.plot(sub_sorted["Date"], sub_sorted["Mean_Direction_deg"],
                     label=f"{utc_label}", marker='x', linestyle='-')
    plt.title(f"Mean Wind Direction Above {thr} knots Over Time (1992–2014)")
    plt.ylabel("Mean Direction (degrees)")
    plt.xlabel("Date")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

# Print overall averages
for thr in thresholds:
    for utc_label in ["3UTC", "12UTC"]:
        sub = df[
            (df["Threshold_knots"] == thr) &
            (df["UTC_Label"] == utc_label)
        ]
        if not sub.empty:
            mean_speed_knots = sub["Mean_Speed_knots"].mean()
            mean_dir = sub["Mean_Direction_deg"].mean()
            mean_area = sub["Area_deg2"].mean()
            print(f"\n🌟 {utc_label} | Threshold {thr} knots:")
            print(f"▪ Mean Speed: {mean_speed_knots:.2f} knots")
            print(f"▪ Mean Direction: {mean_dir:.2f}°")
            print(f"▪ Mean Area: {mean_area:.2f} deg²")


In [ ]:
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

# Upload the CSV file
uploaded = files.upload()

# Load the CSV file
input_csv = "(1992-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)
print("📊 DataFrame shape:", df.shape)
print("🧾 Columns:", df.columns)

# Filter for threshold 20 knots
thr = 20
df = df[df["Threshold_knots"] == thr]

# Add 'Year' column for grouping
df["Year"] = df["Date"].dt.year
years = sorted(df["Year"].unique())

# Plot area over time for each year
for year in years:
    df_year = df[df["Year"] == year]
    plt.figure(figsize=(12, 5))
    for utc_label in ["3UTC", "12UTC"]:
        sub = df_year[df_year["UTC_Label"] == utc_label].sort_values("Date")
        if not sub.empty:
            plt.plot(sub["Date"], sub["Area_deg2"],
                     label=f"{utc_label}", marker='o', linestyle='-')
    plt.title(f"[{year}] Area Above 20 knots")
    plt.ylabel("Area (deg²)")
    plt.xlabel("Date")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

# Plot mean wind direction over time for each year
for year in years:
    df_year = df[df["Year"] == year]
    plt.figure(figsize=(12, 5))
    for utc_label in ["3UTC", "12UTC"]:
        sub = df_year[df_year["UTC_Label"] == utc_label].sort_values("Date")
        if not sub.empty:
            plt.plot(sub["Date"], sub["Mean_Direction_deg"],
                     label=f"{utc_label}", marker='x', linestyle='-')
    plt.title(f"[{year}] Wind Direction Above 20 knots")
    plt.ylabel("Direction (°)")
    plt.xlabel("Date")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
from google.colab import files

# Upload CSV
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Read CSV
df = pd.read_csv(file_name)
print("✅ CSV loaded:", file_name)
print(df.head())  # Show first few rows
print(df.columns)  # Show column names



In [ ]:
import matplotlib.pyplot as plt

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for 2002 and 2003
df_2002 = df[df['Date'].dt.year == 2002]
df_2003 = df[df['Date'].dt.year == 2003]

# Plot Area_deg2 over time
plt.figure(figsize=(15, 6))
plt.plot(df_2002['Date'], df_2002['Area_deg2'], label='2002', color='blue', alpha=0.7)
plt.plot(df_2003['Date'], df_2003['Area_deg2'], label='2003', color='red', alpha=0.7)

plt.title("📈 Area (deg²) above 20 knots — 2002 vs 2003")
plt.ylabel("Area (deg²)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

# Upload the CSV file
uploaded = files.upload()

# Load data
input_csv = "(1992-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)

# Extract year and decade columns
df["Year"] = df["Date"].dt.year
df["Decade"] = (df["Year"] // 10) * 10

# Thresholds to analyze
thresholds = [20, 30, 40]
utc_labels = ["3UTC", "12UTC"]
stats_cols = ["Mean_Speed_knots", "Mean_Direction_deg", "Area_deg2"]

# Store statistics
from collections import defaultdict
stats_summary = defaultdict(dict)

# === YEARLY AND DECADAL AGGREGATES + VARIANCES ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]

        if subset.empty:
            continue

        # Yearly statistics
        yearly_stats = subset.groupby("Year")[stats_cols].agg(["mean", "var"])
        print(f"\n📅 YEARLY STATS for {thr} knots, {utc}")
        print(yearly_stats)

        # Plot yearly changes
        for col in stats_cols:
            plt.figure(figsize=(10, 4))
            plt.plot(yearly_stats.index, yearly_stats[(col, "mean")], marker='o', label="Mean")
            plt.title(f"{col} Yearly Mean - {thr} knots {utc}")
            plt.xlabel("Year")
            plt.ylabel(col)
            plt.grid()
            plt.tight_layout()
            plt.show()

            plt.figure(figsize=(10, 4))
            plt.plot(yearly_stats.index, yearly_stats[(col, "var")], marker='x', label="Variance", color='red')
            plt.title(f"{col} Yearly Variance - {thr} knots {utc}")
            plt.xlabel("Year")
            plt.ylabel(f"Variance of {col}")
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Decadal statistics
        decadal_stats = subset.groupby("Decade")[stats_cols].agg(["mean", "var"])
        print(f"\n🧮 DECADAL STATS for {thr} knots, {utc}")
        print(decadal_stats)

        # Plot decadal changes
        for col in stats_cols:
            plt.figure(figsize=(8, 4))
            plt.plot(decadal_stats.index, decadal_stats[(col, "mean")], marker='s', label="Mean")
            plt.title(f"{col} Decadal Mean - {thr} knots {utc}")
            plt.xlabel("Decade")
            plt.ylabel(col)
            plt.grid()
            plt.tight_layout()
            plt.show()

            plt.figure(figsize=(8, 4))
            plt.plot(decadal_stats.index, decadal_stats[(col, "var")], marker='^', color='purple', label="Variance")
            plt.title(f"{col} Decadal Variance - {thr} knots {utc}")
            plt.xlabel("Decade")
            plt.ylabel(f"Variance of {col}")
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Overall variance for this threshold and UTC label
        overall_variance = subset[stats_cols].var()
        stats_summary[(thr, utc)]["Overall Variance"] = overall_variance

# === OVERALL VARIANCE ACROSS FULL DATASET ===
print("\n📈 OVERALL VARIANCE (entire dataset per threshold and UTC):")
for key, val in stats_summary.items():
    thr, utc = key
    print(f"\n⚡ {thr} knots | {utc}:")
    print(val["Overall Variance"])


In [ ]:
# === Step 0: Imports and Upload CSV ===
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from collections import defaultdict

# Upload your CSV file
uploaded = files.upload()

# === Step 1: Load and Preprocess ===
input_csv = "(1963-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)

df["Year"] = df["Date"].dt.year
df["Decade"] = (df["Year"] // 10) * 10
df["Month"] = df["Date"].dt.month

# Filter JJAS months
df_jjas = df[df["Month"].isin([6, 7, 8, 9])]

# Define thresholds and labels
thresholds = [20, 30, 40]
utc_labels = ["3UTC", "12UTC"]
stats_cols = ["Mean_Speed_knots", "Mean_Direction_deg", "Area_deg2"]

# Store summary
stats_summary = defaultdict(dict)

# === Step 2: Monthly and Seasonal Plots ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df_jjas[(df_jjas["Threshold_knots"] == thr) & (df_jjas["UTC_Label"] == utc)]
        if subset.empty:
            continue

        print(f"\n=== {thr} knots | {utc} ===")

        # Monthly Averages
        monthly_avg = subset.groupby("Month")[stats_cols].mean()

        # A. Monthly Wind Speed Variation (Bar Chart)
        plt.figure(figsize=(8, 5))
        plt.bar(monthly_avg.index, monthly_avg["Mean_Speed_knots"], color='skyblue', edgecolor='black')
        plt.title(f"Monthly Mean Wind Speed (JJAS) - {thr} knots {utc}")
        plt.xlabel("Month")
        plt.ylabel("Mean Wind Speed (knots)")
        plt.xticks([6, 7, 8, 9], ["June", "July", "Aug", "Sept"])
        plt.grid(axis='y')
        plt.tight_layout()
        plt.show()

        # Regular monthly line plots
        for col in stats_cols:
            plt.figure(figsize=(8, 4))
            plt.plot(monthly_avg.index, monthly_avg[col], marker='o')
            plt.title(f"{col} Monthly Avg (JJAS) - {thr} knots {utc}")
            plt.xlabel("Month")
            plt.ylabel(col)
            plt.xticks([6, 7, 8, 9], ["June", "July", "Aug", "Sept"])
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Seasonal Boxplots (per year)
        for col in stats_cols:
            season_data = subset.groupby("Year")[col].apply(list).dropna()

            plt.figure(figsize=(16, 6))  # Enlarged size
            plt.boxplot(season_data,
                        positions=season_data.index,
                        widths=0.8,
                        patch_artist=True,
                        boxprops=dict(facecolor='lightblue'),
                        medianprops=dict(color='red', linewidth=2),
                        whiskerprops=dict(linewidth=1.5),
                        capprops=dict(linewidth=1.5),
                        flierprops=dict(marker='o', markersize=4, markerfacecolor='gray'))

            plt.title(f"Seasonal Boxplot of {col} (JJAS) - {thr} knots {utc}", fontsize=14)
            plt.xlabel("Year")
            plt.ylabel(col)
            plt.grid()
            plt.tight_layout()
            plt.show()

# === Step 3: Spectral (FFT) Analysis ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if subset.empty:
            continue

        for col in stats_cols:
            yearly_mean = subset.groupby("Year")[col].mean().dropna()
            if yearly_mean.empty or len(yearly_mean) < 10:
                continue

            signal = yearly_mean.values
            N = len(signal)
            T = 1  # 1 year interval
            freqs = fftfreq(N, T)
            fft_vals = fft(signal)

            power = np.abs(fft_vals[:N//2]) ** 2
            freqs = freqs[:N//2]

            plt.figure(figsize=(10, 4))
            plt.plot(freqs, power)
            plt.title(f"Power Spectrum of {col} - {thr} knots {utc}")
            plt.xlabel("Frequency (1/year)")
            plt.ylabel("Power")
            plt.grid()
            plt.tight_layout()
            plt.show()

            # Dominant Oscillation
            dominant_freq = freqs[np.argmax(power)]
            period = 1 / dominant_freq if dominant_freq != 0 else float('inf')
            print(f"🔁 Dominant Period for {col} ({thr} knots, {utc}): ~{period:.2f} years")

# === Step 4: Optional Overall Variance Report ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if not subset.empty:
            stats_summary[(thr, utc)]["Overall Variance"] = subset[stats_cols].var()

print("\n📈 OVERALL VARIANCE (entire dataset per threshold and UTC):")
for key, val in stats_summary.items():
    thr, utc = key
    print(f"\n⚡ {thr} knots | {utc}:")
    print(val["Overall Variance"])


In [ ]:
# === Step 0: Imports and Upload CSV ===
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from collections import defaultdict

# Upload your CSV file
uploaded = files.upload()

# === Step 1: Load and Preprocess ===
input_csv = "(1963-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)

df["Year"] = df["Date"].dt.year
df["Decade"] = (df["Year"] // 10) * 10
df["Month"] = df["Date"].dt.month

# Filter JJAS months
df_jjas = df[df["Month"].isin([6, 7, 8, 9])]

# Define thresholds and labels
thresholds = [20, 30, 40]
utc_labels = ["3UTC", "12UTC"]
stats_cols = ["Mean_Speed_knots", "Mean_Direction_deg", "Area_deg2"]

# Store summary
stats_summary = defaultdict(dict)

# === Step 2: Monthly and Seasonal Plots ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df_jjas[(df_jjas["Threshold_knots"] == thr) & (df_jjas["UTC_Label"] == utc)]
        if subset.empty:
            continue

        print(f"\n=== {thr} knots | {utc} ===")

        # Monthly Averages
        monthly_avg = subset.groupby("Month")[stats_cols].mean()

        # A. Monthly Wind Speed Variation (Bar Chart)
        plt.figure(figsize=(8, 5))
        plt.bar(monthly_avg.index, monthly_avg["Mean_Speed_knots"], color='skyblue', edgecolor='black')
        plt.title(f"Monthly Mean Wind Speed (JJAS) - {thr} knots {utc}")
        plt.xlabel("Month")
        plt.ylabel("Mean Wind Speed (knots)")
        plt.xticks([6, 7, 8, 9], ["June", "July", "Aug", "Sept"])
        plt.grid(axis='y')
        plt.tight_layout()
        plt.show()

        # Monthly Line Plots
        for col in stats_cols:
            plt.figure(figsize=(8, 4))
            plt.plot(monthly_avg.index, monthly_avg[col], marker='o')
            plt.title(f"{col} Monthly Avg (JJAS) - {thr} knots {utc}")
            plt.xlabel("Month")
            plt.ylabel(col)
            plt.xticks([6, 7, 8, 9], ["June", "July", "Aug", "Sept"])
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Seasonal Boxplots with line graph + trend line
        for col in stats_cols:
            season_data = subset.groupby("Year")[col].apply(list).dropna()
            yearly_means = subset.groupby("Year")[col].mean().dropna()
            years = yearly_means.index
            means = yearly_means.values

            plt.figure(figsize=(16, 6))
            # Boxplot
            plt.boxplot(season_data,
                        positions=season_data.index,
                        widths=0.8,
                        patch_artist=True,
                        boxprops=dict(facecolor='lightblue'),
                        medianprops=dict(color='red', linewidth=2),
                        whiskerprops=dict(linewidth=1.5),
                        capprops=dict(linewidth=1.5),
                        flierprops=dict(marker='o', markersize=4, markerfacecolor='gray'))

            # Line plot of means
            plt.plot(years, means, color='black', marker='o', linestyle='-', label='Yearly Mean')

            # Trend line
            z = np.polyfit(years, means, 1)
            p = np.poly1d(z)
            plt.plot(years, p(years), color='darkblue', linewidth=2, linestyle='--', label='Trend')

            plt.title(f"{col} Seasonal Boxplot + Mean Line (JJAS) - {thr} knots {utc}", fontsize=14)
            plt.xlabel("Year")
            plt.ylabel(col)
            plt.legend()
            plt.grid()
            plt.tight_layout()
            plt.show()

# === Step 3: Spectral (FFT) Analysis — Only Print Period ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if subset.empty:
            continue

        for col in stats_cols:
            yearly_mean = subset.groupby("Year")[col].mean().dropna()
            if yearly_mean.empty or len(yearly_mean) < 10:
                continue

            signal = yearly_mean.values - np.mean(yearly_mean.values)
            N = len(signal)
            T = 1  # 1 year interval
            freqs = fftfreq(N, T)
            fft_vals = fft(signal)

            power = np.abs(fft_vals[:N//2]) ** 2
            freqs = freqs[:N//2]

            nonzero_freqs = freqs[freqs > 0]
            nonzero_power = power[freqs > 0]
            if len(nonzero_freqs) > 0:
                dominant_freq = nonzero_freqs[np.argmax(nonzero_power)]
                period = 1 / dominant_freq
            else:
                period = float('inf')

            print(f"🔁 Dominant Period for {col} ({thr} knots, {utc}): ~{period:.2f} years")

# === Step 4: Overall Variance Report ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if not subset.empty:
            stats_summary[(thr, utc)]["Overall Variance"] = subset[stats_cols].var()

print("\n📈 OVERALL VARIANCE (entire dataset per threshold and UTC):")
for key, val in stats_summary.items():
    thr, utc = key
    print(f"\n⚡ {thr} knots | {utc}:")
    print(val["Overall Variance"])


In [ ]:
# === Step 0: Imports and Upload CSV ===
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq

# === Upload CSV File ===
uploaded = files.upload()
input_csv = "(1963-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)

# === Step 1: Preprocess ===
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.dayofyear  # For daily grouping

# Filter JJAS months
df_jjas = df[df["Month"].isin([6, 7, 8, 9])]

# Define parameters
thresholds = [20, 30, 40]
utc_labels = ["3UTC", "12UTC"]
stats_cols = ["Mean_Speed_knots", "Mean_Direction_deg", "Area_deg2"]

# === Step 2: Power Spectrum Analysis with Bar Plots ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df_jjas[(df_jjas["Threshold_knots"] == thr) & (df_jjas["UTC_Label"] == utc)]
        if subset.empty:
            continue

        print(f"\n🌊 FFT for {thr} knots | {utc}")

        for col in stats_cols:
            # === A. Yearly Average FFT ===
            yearly_avg = subset.groupby("Year")[col].mean().dropna()
            if len(yearly_avg) >= 10:
                signal = yearly_avg.values - np.mean(yearly_avg.values)
                N = len(signal)
                T = 1  # Yearly interval
                freqs = fftfreq(N, T)
                fft_vals = fft(signal)
                power = np.abs(fft_vals[:N // 2]) ** 2
                freqs = freqs[:N // 2]

                plt.figure(figsize=(10, 4))
                plt.bar(freqs, power, width=0.01, color='steelblue', edgecolor='black')
                plt.title(f"Power Spectrum (Yearly Avg) of {col} - {thr} knots {utc}")
                plt.xlabel("Frequency (1/year)")
                plt.ylabel("Power")
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.tight_layout()
                plt.show()

            # === B. Daily Average Across All Years FFT ===
            daily_avg = subset.groupby("Day")[col].mean().dropna()
            if len(daily_avg) >= 50:
                signal = daily_avg.values - np.mean(daily_avg.values)
                N = len(signal)
                T = 1  # Daily interval
                freqs = fftfreq(N, T)
                fft_vals = fft(signal)
                power = np.abs(fft_vals[:N // 2]) ** 2
                freqs = freqs[:N // 2]

                plt.figure(figsize=(10, 4))
                plt.bar(freqs, power, width=0.002, color='darkorange', edgecolor='black')
                plt.title(f"Power Spectrum (Daily Avg) of {col} - {thr} knots {utc}")
                plt.xlabel("Frequency (1/day)")
                plt.ylabel("Power")
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.tight_layout()
                plt.show()


In [ ]:
!pip install windrose


In [ ]:
from windrose import WindroseAxes
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Filter JJAS months from original DataFrame
df_jjas = df[df["Month"].isin([6, 7, 8, 9])]

# Choose one UTC (or loop through both if needed)
utc = "3UTC"  # or "12UTC"
thr = 20  # or any threshold value

# Subset for specific UTC and threshold
subset = df_jjas[(df_jjas["UTC_Label"] == utc) & (df_jjas["Threshold_knots"] == thr)]
if subset.empty:
    print("No data found for the selected threshold and UTC.")
else:
    print(f"🌬️ Wind Rose for JJAS | {thr} knots | {utc} | Entries: {len(subset)}")

    # Wind speed and direction arrays
    wind_speed = subset["Mean_Speed_knots"].values
    wind_dir = subset["Mean_Direction_deg"].values

    # --- Create Wind Rose Plot ---
    ax = WindroseAxes.from_ax()
    ax.bar(wind_dir, wind_speed,
           normed=True,
           opening=0.8,
           edgecolor='white',
           bins=np.arange(0, 60, 5),  # speed bins
           cmap=cm.viridis)

    ax.set_legend(title="Wind Speed (knots)")
    plt.title(f"JJAS Wind Rose ({thr} knots, {utc})")
    plt.tight_layout()
    plt.show()

    # --- Compute Dominant Wind Direction ---
    # Bin wind direction into 8 or 16 compass sectors
    dir_bins = np.arange(0, 360 + 22.5, 22.5)  # 16 sectors
    dir_labels = [(dir_bins[i] + dir_bins[i+1])/2 for i in range(len(dir_bins)-1)]
    binned_dirs = pd.cut(wind_dir, bins=dir_bins, labels=dir_labels, include_lowest=True)

    # Most common direction
    dominant_dir = binned_dirs.value_counts().idxmax()
    print(f"🧭 Dominant Wind Direction (°): {dominant_dir}")

    # --- Compute Dominant Speed Bin ---
    speed_bins = pd.cut(wind_speed, bins=np.arange(0, 60, 5))
    dominant_speed = speed_bins.value_counts().idxmax()
    print(f"⚡ Dominant Wind Speed Range: {dominant_speed}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from windrose import WindroseAxes
import matplotlib.gridspec as gridspec

# Load and prepare data
df = pd.read_csv("(1963-2024).csv")  # Replace with your file
df['Date'] = pd.to_datetime(df['Date'])

# Filter JJAS only
df_jjas = df[df['Date'].dt.month.isin([6, 7, 8, 9])]
df_jjas['Year'] = df_jjas['Date'].dt.year

# Extract unique years
years = sorted(df_jjas['Year'].unique())
n_years = len(years)

# Layout: Calculate rows and cols for subplots
ncols = 3
nrows = -(-n_years // ncols)  # Ceiling division

# Create figure
fig = plt.figure(figsize=(5 * ncols, 5 * nrows))
gs = gridspec.GridSpec(nrows, ncols)

for i, year in enumerate(years):
    row, col = divmod(i, ncols)
    ax = WindroseAxes(fig, gs[row, col])
    fig.add_axes(ax)

    df_year = df_jjas[df_jjas['Year'] == year]
    wind_speed = df_year['Mean_Speed_knots']
    wind_dir = df_year['Mean_Direction_deg']

    ax.bar(wind_dir, wind_speed, normed=True, opening=0.8, edgecolor='white',
           bins=[0, 5, 10, 15, 20, 25])
    ax.set_title(f"JJAS {year}", fontsize=12)

# Adjust layout and add legend
plt.tight_layout()
fig.legend(*ax.get_legend_handles_labels(), loc='lower center', ncol=6, fontsize=10)
plt.suptitle("Wind Rose Plots (JJAS) for Each Year", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92, bottom=0.1)
plt.show()


In [ ]:
# 📦 Install tabulate if not already installed
!pip install tabulate

import pandas as pd
import numpy as np
from tabulate import tabulate

# 📂 Load your dataset (replace 'your_file.csv' accordingly)
# df = pd.read_csv('your_file.csv')

# 🧼 Preprocessing
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# 🌦️ Filter JJAS (June to September)
jjas_df = df[df['Month'].isin([6, 7, 8, 9])].copy()

# 🧭 Bin wind direction into 8 cardinal categories (N, NE, E, SE, S, SW, W, NW)
def direction_category(deg):
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    bins = np.arange(-22.5, 361, 45)
    idx = np.digitize([deg], bins, right=True)[0] % 8
    return directions[idx]

jjas_df['Direction_Category'] = jjas_df['Mean_Direction_deg'].apply(direction_category)

# ⚡ Bin wind speed
bins = [0, 5, 10, 15, 20, 25, 30, 100]
labels = ['(0-5]', '(5-10]', '(10-15]', '(15-20]', '(20-25]', '(25-30]', '(30+)']
jjas_df['Speed_Range'] = pd.cut(jjas_df['Mean_Speed_knots'], bins=bins, labels=labels, right=True)

# 🧮 Group by year, direction, and speed
grouped = jjas_df.groupby(['Year', 'Direction_Category', 'Speed_Range']).size().reset_index(name='Frequency')

# 🔍 Find dominant direction-speed combo per year
dominant = grouped.loc[grouped.groupby('Year')['Frequency'].idxmax()].reset_index(drop=True)

# 🪄 Rename for clarity
dominant.rename(columns={
    'Direction_Category': 'Dominant_Direction',
    'Speed_Range': 'Dominant_Speed_Range'
}, inplace=True)

# 📊 Display tabulated output
print(tabulate(dominant, headers='keys', tablefmt='github', showindex=False))



In [ ]:
# 📦 Import necessary libraries
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from google.colab import files

# 📂 Upload rainfall file
print("📂 Upload the Rainfall CSV file:")
uploaded_rain = files.upload()
rainfall_file = list(uploaded_rain.keys())[0]
df_rain = pd.read_csv(rainfall_file)

# 📂 Upload wind speed file
print("\n📂 Now upload the Wind Speed CSV file:")
uploaded_wind = files.upload()
wind_file = list(uploaded_wind.keys())[0]
df_wind = pd.read_csv(wind_file)

# 🧹 Process wind file: extract 'year' from 'Date' and group by year
df_wind['Date'] = pd.to_datetime(df_wind['Date'])
df_wind['year'] = df_wind['Date'].dt.year
wind_yearly = df_wind.groupby('year')['Mean_Speed_knots'].mean().reset_index()
wind_yearly.rename(columns={'Mean_Speed_knots': 'WIND_SPEED'}, inplace=True)

# 🧹 Process rainfall file: group by 'year'
rain_yearly = df_rain.groupby('year')['RAINFALL'].mean().reset_index()

# 🔗 Merge the two yearly summaries on 'year'
merged = pd.merge(rain_yearly, wind_yearly, on='year', how='inner')

# ✅ Drop missing values
cleaned = merged.dropna(subset=["RAINFALL", "WIND_SPEED"])

# ============================================
# 📊 CORRELATION ANALYSIS
# ============================================
pearson_corr, _ = pearsonr(cleaned['RAINFALL'], cleaned['WIND_SPEED'])
spearman_corr, _ = spearmanr(cleaned['RAINFALL'], cleaned['WIND_SPEED'])

print("\n✅ Correlation Results:")
print(f"📈 Pearson Correlation: {pearson_corr:.4f}")
print(f"📉 Spearman Correlation: {spearman_corr:.4f}")



In [ ]:
# 📦 Import necessary libraries
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
from google.colab import files

# 📂 Upload Rainfall CSV file
print("📂 Upload Rainfall CSV (with columns like time, RAINFALL):")
uploaded_rain = files.upload()
rainfall_file = list(uploaded_rain.keys())[0]
df_rain = pd.read_csv(rainfall_file)

# 📂 Upload Wind Speed CSV file
print("\n📂 Upload Wind Speed CSV (with Date, Mean_Speed_knots):")
uploaded_wind = files.upload()
wind_file = list(uploaded_wind.keys())[0]
df_wind = pd.read_csv(wind_file)

# 🧹 Convert 'Date' column to datetime in both
df_rain['Date'] = pd.to_datetime(df_rain['time'])  # Replace 'time' if your column is named differently
df_wind['Date'] = pd.to_datetime(df_wind['Date'])

# 📆 Add 'year' columns
df_rain['year'] = df_rain['Date'].dt.year
df_wind['year'] = df_wind['Date'].dt.year

# 📊 Merge on 'Date'
merged_df = pd.merge(
    df_rain[['Date', 'RAINFALL', 'year']],
    df_wind[['Date', 'Mean_Speed_knots']],
    on='Date', how='inner'
)

# Rename for clarity
merged_df.rename(columns={'Mean_Speed_knots': 'WIND_SPEED'}, inplace=True)

# 📈 Calculate correlation for each year
results = []

for year, group in merged_df.groupby('year'):
    if len(group) > 10:  # Avoid very small sample sizes
        r, _ = pearsonr(group['RAINFALL'], group['WIND_SPEED'])
        s, _ = spearmanr(group['RAINFALL'], group['WIND_SPEED'])
        results.append({'year': year, 'pearson_corr': r, 'spearman_corr': s})
    else:
        results.append({'year': year, 'pearson_corr': None, 'spearman_corr': None})

# 📄 Convert to DataFrame
correlation_df = pd.DataFrame(results)

# ✅ Option 1: Clean print using to_string
print("\n📊 Year-wise Correlation Analysis (using pandas to_string):")
print(correlation_df.to_string(index=False, float_format="%.6f"))

# ✅ Option 2: Pretty table using tabulate (optional)
try:
    from tabulate import tabulate
    print("\n📊 Year-wise Correlation Analysis (using tabulate):")
    print(tabulate(correlation_df, headers='keys', tablefmt='pretty', floatfmt=".6f"))
except ImportError:
    print("\n🔧 tabulate not installed. You can install it with: !pip install tabulate")

# 📉 Plotting correlation trends
plt.figure(figsize=(10, 6))
plt.plot(correlation_df['year'], correlation_df['pearson_corr'], marker='o', label='Pearson Correlation', color='blue')
plt.plot(correlation_df['year'], correlation_df['spearman_corr'], marker='s', label='Spearman Correlation', color='green')

plt.title('Yearly Rainfall-Wind Correlation Trends')
plt.xlabel('Year')
plt.ylabel('Correlation Coefficient')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# 📦 Import necessary libraries
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from google.colab import files

# 📂 Upload Rainfall CSV file
print("📂 Upload Rainfall CSV (with columns like time, RAINFALL):")
uploaded_rain = files.upload()
rainfall_file = list(uploaded_rain.keys())[0]
df_rain = pd.read_csv(rainfall_file)

# 📂 Upload Wind Speed CSV file
print("\n📂 Upload Wind Speed CSV (with Date, Mean_Speed_knots):")
uploaded_wind = files.upload()
wind_file = list(uploaded_wind.keys())[0]
df_wind = pd.read_csv(wind_file)

# 🧹 Convert 'Date' column to datetime in both
df_rain['Date'] = pd.to_datetime(df_rain['time'])  # Replace 'time' if your column is named differently
df_wind['Date'] = pd.to_datetime(df_wind['Date'])

# 📆 Add 'year' columns
df_rain['year'] = df_rain['Date'].dt.year
df_wind['year'] = df_wind['Date'].dt.year

# 📊 Merge on 'Date'
merged_df = pd.merge(
    df_rain[['Date', 'RAINFALL', 'year']],
    df_wind[['Date', 'Mean_Speed_knots']],
    on='Date', how='inner'
)

# Rename for clarity
merged_df.rename(columns={'Mean_Speed_knots': 'WIND_SPEED'}, inplace=True)

# 🌀 Function to calculate lagged correlations
def calculate_lag_correlations(df, max_lag=3):
    results = []
    for year, group in df.groupby('year'):
        if len(group) > max_lag + 10:
            row = {'year': year}
            for lag in range(-max_lag, max_lag + 1):  # Negative to positive lags
                shifted = group.copy()
                shifted['WIND_SPEED_LAG'] = shifted['WIND_SPEED'].shift(lag)
                valid = shifted.dropna()

                if len(valid) > 10:
                    pearson_corr, _ = pearsonr(valid['RAINFALL'], valid['WIND_SPEED_LAG'])
                    spearman_corr, _ = spearmanr(valid['RAINFALL'], valid['WIND_SPEED_LAG'])
                    row[f'pearson_lag_{lag}'] = pearson_corr
                    row[f'spearman_lag_{lag}'] = spearman_corr
                else:
                    row[f'pearson_lag_{lag}'] = None
                    row[f'spearman_lag_{lag}'] = None
            results.append(row)
    return pd.DataFrame(results)

# 🍃 Filter only monsoon months (June to September)
merged_df = merged_df[merged_df['Date'].dt.month.isin([6, 7, 8, 9])]

# 📈 Now calculate lagged correlation for filtered data
correlation_df = calculate_lag_correlations(merged_df, max_lag=3)


# 📋 Reorder columns: first all Pearson lags, then all Spearman lags
pearson_cols = [col for col in correlation_df.columns if col.startswith('pearson_lag_')]
spearman_cols = [col for col in correlation_df.columns if col.startswith('spearman_lag_')]
ordered_cols = ['year'] + sorted(pearson_cols, key=lambda x: int(x.split('_')[-1])) + sorted(spearman_cols, key=lambda x: int(x.split('_')[-1]))
correlation_df = correlation_df[ordered_cols]

# ✅ Option 1: Clean print using to_string
print("\n📊 Year-wise Lag Correlation Analysis (using pandas to_string):")
print(correlation_df.to_string(index=False, float_format="%.6f"))

# ✅ Option 2: Pretty table using tabulate (optional)
try:
    from tabulate import tabulate
    print("\n📊 Year-wise Lag Correlation Analysis (using tabulate):")
    print(tabulate(correlation_df, headers='keys', tablefmt='pretty', floatfmt=".6f"))
except ImportError:
    print("\n🔧 tabulate not installed. You can install it with: !pip install tabulate")

# 📊 Plotting setup
import matplotlib.pyplot as plt
import seaborn as sns

# 🌈 Convert DataFrame to long format for heatmap
def prepare_heatmap_data(df, method='pearson'):
    heatmap_df = df.copy()
    id_vars = ['year']
    value_vars = [col for col in df.columns if col.startswith(f'{method}_lag_')]

    melted = pd.melt(heatmap_df, id_vars=id_vars, value_vars=value_vars,
                     var_name='lag', value_name='correlation')

    # Extract numeric lag values
    melted['lag'] = melted['lag'].str.extract(r'lag_(-?\d+)').astype(int)

    # Pivot to make heatmap
    pivot_df = melted.pivot(index='year', columns='lag', values='correlation')
    return pivot_df

# 🔥 Plot heatmaps
for method in ['pearson', 'spearman']:
    heatmap_data = prepare_heatmap_data(correlation_df, method=method)

    plt.figure(figsize=(10, 6))
    sns.heatmap(heatmap_data, annot=True, center=0, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title(f'{method.capitalize()} Lag Correlation: Rainfall vs Wind Speed')
    plt.xlabel("Lag (days)")
    plt.ylabel("Year")
    plt.tight_layout()
    plt.show()

# 📊 Bar plot of average lag correlation across years
def plot_avg_lag_correlation(df, method='pearson'):
    avg_corr = {}
    for col in df.columns:
        if col.startswith(f'{method}_lag_'):
            avg_corr[col] = df[col].mean()

    # Convert to sorted DataFrame
    lag_df = pd.DataFrame({
        'Lag': [int(k.split('_lag_')[1]) for k in avg_corr.keys()],
        'Avg_Correlation': list(avg_corr.values())
    }).sort_values('Lag')

    plt.figure(figsize=(8, 5))
    sns.barplot(data=lag_df, x='Lag', y='Avg_Correlation', palette='coolwarm')
    plt.title(f"Average {method.capitalize()} Lag Correlation (All Years)")
    plt.axhline(0, color='black', linestyle='--')
    plt.xlabel("Lag (days)")
    plt.ylabel("Average Correlation")
    plt.tight_layout()
    plt.show()

# 📊 Average correlation bar plots
plot_avg_lag_correlation(correlation_df, method='pearson')
plot_avg_lag_correlation(correlation_df, method='spearman')

# 📈 Line plot of lag correlation per year
def plot_lag_correlation_lines(df, method='pearson'):
    plt.figure(figsize=(10, 6))

    for _, row in df.iterrows():
        year = row['year']
        lags = []
        values = []
        for col in row.index:
            if col.startswith(f'{method}_lag_'):
                lag = int(col.split('_lag_')[1])
                lags.append(lag)
                values.append(row[col])
        plt.plot(lags, values, marker='o', label=str(year))

    plt.title(f"{method.capitalize()} Lag Correlation by Year")
    plt.xlabel("Lag (days)")
    plt.ylabel("Correlation")
    plt.axhline(0, color='gray', linestyle='--')
    plt.grid(True)
    plt.legend(title="Year")
    plt.tight_layout()
    plt.show()

# 📈 Line plots
plot_lag_correlation_lines(correlation_df, method='pearson')
plot_lag_correlation_lines(correlation_df, method='spearman')


In [ ]:
# 📦 Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

# 📂 Upload Rainfall CSV file
print("📂 Upload Rainfall CSV (with columns like time, RAINFALL):")
uploaded_rain = files.upload()
rainfall_file = list(uploaded_rain.keys())[0]
df_rain = pd.read_csv(rainfall_file)

# 📂 Upload Wind Speed CSV file
print("\n📂 Upload Wind Speed CSV (with Date, Mean_Speed_knots):")
uploaded_wind = files.upload()
wind_file = list(uploaded_wind.keys())[0]
df_wind = pd.read_csv(wind_file)

# 🧹 Convert date columns to datetime format
df_rain['Date'] = pd.to_datetime(df_rain['time'])  # Adjust if column name differs
df_wind['Date'] = pd.to_datetime(df_wind['Date'])

# 📊 Merge datasets on 'Date'
merged_df = pd.merge(
    df_rain[['Date', 'RAINFALL']],
    df_wind[['Date', 'Mean_Speed_knots']],
    on='Date', how='inner'
)

# ✅ Rename for clarity
merged_df.rename(columns={'Mean_Speed_knots': 'WIND_SPEED'}, inplace=True)

# 📈 Plot time-series comparison
plt.figure(figsize=(14, 6))

# Plot Rainfall
plt.plot(merged_df['Date'], merged_df['RAINFALL'], label='Rainfall (mm)', color='blue', alpha=0.6)

# Plot Wind Speed on secondary y-axis
plt.plot(merged_df['Date'], merged_df['WIND_SPEED'], label='Wind Speed (knots)', color='green', alpha=0.6)

# 🖼️ Final touches
plt.title('📈 Time-Series: Rainfall vs Wind Speed')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# === Step 0: Imports and Upload CSV ===
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from collections import defaultdict

# Upload your CSV file
uploaded = files.upload()

# === Step 1: Load and Preprocess ===
input_csv = "(1963-2024).csv"
df = pd.read_csv(input_csv, parse_dates=["Date"])
print("✅ Loaded data from", input_csv)

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Decade"] = (df["Year"] // 10) * 10

# Define thresholds and labels
thresholds = [20, 30, 40]
utc_labels = ["3UTC", "12UTC"]
stats_cols = ["Mean_Speed_knots", "Mean_Direction_deg", "Area_deg2"]

# Store summary
stats_summary = defaultdict(dict)

# === Step 2: Monthly and Seasonal Plots (using all months) ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if subset.empty:
            continue

        print(f"\n=== {thr} knots | {utc} ===")

        # Monthly Averages
        monthly_avg = subset.groupby("Month")[stats_cols].mean()

        # A. Monthly Wind Speed Variation (Bar Chart)
        plt.figure(figsize=(8, 5))
        plt.bar(monthly_avg.index, monthly_avg["Mean_Speed_knots"], color='skyblue', edgecolor='black')
        plt.title(f"Monthly Mean Wind Speed (All Months) - {thr} knots {utc}")
        plt.xlabel("Month")
        plt.ylabel("Mean Wind Speed (knots)")
        plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"])
        plt.grid(axis='y')
        plt.tight_layout()
        plt.show()

        # Regular monthly line plots
        for col in stats_cols:
            plt.figure(figsize=(8, 4))
            plt.plot(monthly_avg.index, monthly_avg[col], marker='o')
            plt.title(f"{col} Monthly Avg (All Months) - {thr} knots {utc}")
            plt.xlabel("Month")
            plt.ylabel(col)
            plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"])
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Seasonal Boxplots (per year)
        for col in stats_cols:
            season_data = subset.groupby("Year")[col].apply(list).dropna()

            plt.figure(figsize=(16, 6))
            plt.boxplot(season_data,
                        positions=season_data.index,
                        widths=0.8,
                        patch_artist=True,
                        boxprops=dict(facecolor='lightblue'),
                        medianprops=dict(color='red', linewidth=2),
                        whiskerprops=dict(linewidth=1.5),
                        capprops=dict(linewidth=1.5),
                        flierprops=dict(marker='o', markersize=4, markerfacecolor='gray'))

            plt.title(f"Seasonal Boxplot of {col} (All Months) - {thr} knots {utc}", fontsize=14)
            plt.xlabel("Year")
            plt.ylabel(col)
            plt.grid()
            plt.tight_layout()
            plt.show()

# === Step 3: Spectral (FFT) Analysis ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if subset.empty:
            continue

        for col in stats_cols:
            yearly_mean = subset.groupby("Year")[col].mean().dropna()
            if yearly_mean.empty or len(yearly_mean) < 10:
                continue

            signal = yearly_mean.values
            N = len(signal)
            T = 1  # 1 year interval
            freqs = fftfreq(N, T)
            fft_vals = fft(signal)

            power = np.abs(fft_vals[:N//2]) ** 2
            freqs = freqs[:N//2]

            plt.figure(figsize=(10, 4))
            plt.plot(freqs, power)
            plt.title(f"Power Spectrum of {col} - {thr} knots {utc}")
            plt.xlabel("Frequency (1/year)")
            plt.ylabel("Power")
            plt.grid()
            plt.tight_layout()
            plt.show()

            # Dominant Oscillation
            dominant_freq = freqs[np.argmax(power)]
            period = 1 / dominant_freq if dominant_freq != 0 else float('inf')
            print(f"🔁 Dominant Period for {col} ({thr} knots, {utc}): ~{period:.2f} years")

# === Step 4: Overall Variance Report ===
for thr in thresholds:
    for utc in utc_labels:
        subset = df[(df["Threshold_knots"] == thr) & (df["UTC_Label"] == utc)]
        if not subset.empty:
            stats_summary[(thr, utc)]["Overall Variance"] = subset[stats_cols].var()

print("\n📈 OVERALL VARIANCE (entire dataset per threshold and UTC):")
for key, val in stats_summary.items():
    thr, utc = key
    print(f"\n⚡ {thr} knots | {utc}:")
    print(val["Overall Variance"])


In [ ]:


# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# --- Step 1: Upload CSV ---
#print("Please upload your rainfall CSV file...")
#uploaded = files.upload()

filename = list(uploaded.keys())[0]

# --- Step 2: Load and prepare data ---
df = pd.read_csv(filename)
df['time'] = pd.to_datetime(df['time'], errors='coerce')
df['RAINFALL'] = pd.to_numeric(df['RAINFALL'], errors='coerce')
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day

print("\nData preview:")
print(df.head())

# --- Step 3: Overall stats ---
overall_mean = df['RAINFALL'].mean()
overall_std = df['RAINFALL'].std()

# --- Step 4: Seasonal stats (per month) ---
monthly_stats = df.groupby('month')['RAINFALL'].agg(['mean','std']).reset_index()

# Merge stats back to dataframe
df = df.merge(monthly_stats, on='month', how='left', suffixes=('', '_month'))

# --- Step 5: Anomaly detection ---
# Seasonal anomaly: deviation from that month's historical mean
df['Deviation'] = df['RAINFALL'] - df['mean']
df['Percent_Deviation'] = (df['Deviation'] / df['mean']) * 100
df['Seasonal_Anomaly'] = ((df['RAINFALL'] > df['mean'] + 2*df['std']) |
                          (df['RAINFALL'] < df['mean'] - 2*df['std']))

# --- Step 6: JJAS (Monsoon) stats ---
df['season'] = np.where(df['month'].isin([6,7,8,9]), 'JJAS', 'Other')
season_stats = df.groupby('season')['RAINFALL'].agg(['mean','std']).reset_index()
df = df.merge(season_stats, on='season', how='left', suffixes=('', '_season'))

df['JJAS_Anomaly'] = np.where(
    df['season'] == 'JJAS',
    ((df['RAINFALL'] > df['mean_season'] + 2*df['std_season']) |
     (df['RAINFALL'] < df['mean_season'] - 2*df['std_season'])),
    False
)

# --- Step 7: Save anomalies ---
seasonal_anomalies = df[df['Seasonal_Anomaly'] | df['JJAS_Anomaly']]
seasonal_anomalies.to_csv("seasonal_rainfall_anomalies.csv", index=False)
print("\nSeasonal anomalies saved to seasonal_rainfall_anomalies.csv")
files.download("seasonal_rainfall_anomalies.csv")

# --- Step 8: Visualization ---
plt.figure(figsize=(14,6))
sns.lineplot(x='time', y='RAINFALL', data=df, label='Rainfall')
sns.scatterplot(x='time', y='RAINFALL', data=seasonal_anomalies, color='red', label='Anomalies')
plt.title("Rainfall with Seasonal & JJAS Anomalies")
plt.axhline(overall_mean, color='green', linestyle='--', label='Overall Mean')
plt.xlabel("Date")
plt.ylabel("Rainfall")
plt.legend()
plt.show()

# --- Step 9: Monthly deviation heatmap ---
plt.figure(figsize=(12,6))
monthly_dev = monthly_stats.set_index('month')['mean'] - overall_mean
sns.heatmap(monthly_dev.to_frame().T, annot=True, cmap='coolwarm', center=0)
plt.title("Monthly Mean Deviation from Overall Mean Rainfall")
plt.show()
